In [ ]:
import pandas as pd
import tkinter as tk
from tkinter import messagebox

# Load the CSV files
ind_batting_vs_aus = pd.read_csv('E:\\Major project\\ind_batting_against_aus.csv')
ind_bowling_vs_aus = pd.read_csv('E:\\Major project\\ind_bowling_against_aus.csv')
aus_batting_vs_ind = pd.read_csv('E:\\Major project\\aus_batting_against_ind.csv')
aus_bowling_vs_ind = pd.read_csv('E:\\Major project\\aus_bowling_against_ind.csv')
stadium_conditions = pd.read_csv('E:\\Major project\\stadium_conditions.csv')

# Handle missing values
ind_batting_vs_aus.fillna(method='ffill', inplace=True)
ind_bowling_vs_aus.fillna(method='ffill', inplace=True)
aus_batting_vs_ind.fillna(method='ffill', inplace=True)
aus_bowling_vs_ind.fillna(method='ffill', inplace=True)
stadium_conditions.fillna(method='ffill', inplace=True)

# Print column names for debugging
print("Indian Batting vs Australia Bowlers Columns:", ind_batting_vs_aus.columns)
print("Indian Bowling vs Australia Batsmen Columns:", ind_bowling_vs_aus.columns)
print("Australian Batting vs India Bowlers Columns:", aus_batting_vs_ind.columns)
print("Australian Bowling vs India Batsmen Columns:", aus_bowling_vs_ind.columns)
print("Stadium Conditions Columns:", stadium_conditions.columns)

# Ensure the relevant columns are numeric
def convert_columns_to_numeric(df, columns):
    for col in columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        else:
            print(f"Warning: Column '{col}' not found in DataFrame")
    return df

ind_batting_vs_aus = convert_columns_to_numeric(ind_batting_vs_aus, ['Avg', 'Strike Rate', 'Runs'])
ind_bowling_vs_aus = convert_columns_to_numeric(ind_bowling_vs_aus, ['Avg', 'Econ'])
aus_batting_vs_ind = convert_columns_to_numeric(aus_batting_vs_ind, ['Avg', 'Strike Rate', 'Runs'])
aus_bowling_vs_ind = convert_columns_to_numeric(aus_bowling_vs_ind, ['Avg', 'Econ'])

# Convert Span to numeric
def convert_span_to_numeric(df):
    if 'Span' in df.columns:
        df['Span'] = df['Span'].apply(lambda x: x.split('-') if isinstance(x, str) else [0, 0])
        df['Span'] = df['Span'].apply(lambda x: int(x[1]) - int(x[0]))
    return df

ind_batting_vs_aus = convert_span_to_numeric(ind_batting_vs_aus)
ind_bowling_vs_aus = convert_span_to_numeric(ind_bowling_vs_aus)
aus_batting_vs_ind = convert_span_to_numeric(aus_batting_vs_ind)
aus_bowling_vs_ind = convert_span_to_numeric(aus_bowling_vs_ind)

# Fill any remaining NaN values with 0
ind_batting_vs_aus.fillna(0, inplace=True)
ind_bowling_vs_aus.fillna(0, inplace=True)
aus_batting_vs_ind.fillna(0, inplace=True)
aus_bowling_vs_ind.fillna(0, inplace=True)

# Function to select the best 11 players
def select_best_11(team_batting, team_bowling, stadium_condition):
    stadium_info = stadium_conditions[stadium_conditions['Stadium Name'] == stadium_condition]
    
    if stadium_info.empty:
        print("Stadium not found in stadium conditions dataset.")
        return pd.DataFrame()
    
    pitch_type = stadium_info.iloc[0]['Pitch Type']
    is_batsman_friendly = stadium_info.iloc[0]['Batsman Friendly (1/0)']
    is_bowler_friendly = stadium_info.iloc[0]['Bowler Friendly (1/0)']
    
    # Print stadium info for debugging
    print(f"Stadium Info: {stadium_info}")
    
    # Add a priority score for batsmen based on average, strike rate, runs, and span
    team_batting['Priority'] = (team_batting['Avg'] * 0.4 + 
                                team_batting['Strike Rate'] * 0.3 + 
                                team_batting['Runs'] * 0.2 + 
                                team_batting['Span'] * 0.1)
    
    # Add a priority score for bowlers based on average, economy, and span
    team_bowling['Priority'] = (team_bowling['Avg'] * 0.4 + 
                                team_bowling['Econ'] * 0.4 + 
                                team_bowling['Span'] * 0.2)
    
    # Sort players based on their priority score
    best_batsmen = team_batting.sort_values(by='Priority', ascending=False)
    best_bowlers = team_bowling.sort_values(by='Priority', ascending=False)
    
    # Select best all-rounders
    all_rounders = team_bowling[team_bowling['Player'].isin(team_batting['Player'])]
    best_all_rounders = all_rounders.sort_values(by='Priority', ascending=False)
    
    # Select top 6 batsmen
    top_batsmen = best_batsmen.head(6)
    
    # Select top 4 bowlers
    top_bowlers = best_bowlers.head(4)
    
    # Select top 2 all-rounders
    top_all_rounders = best_all_rounders.head(2)
    
    # Combine the best players ensuring no duplicates
    selected_team = pd.concat([top_batsmen, top_bowlers, top_all_rounders]).drop_duplicates().head(11)
    
    print(f"Selected Team Data: {selected_team}")
    
    return selected_team

# Function to predict performance
def predict_performance(player_data, batting_data, bowling_data):
    predictions = {}
    for _, player in player_data.iterrows():
        player_name = player['Player']
        predictions[player_name] = {}
        if player_name in batting_data['Player'].values:
            batting_info = batting_data[batting_data['Player'] == player_name]
            predictions[player_name] = {
                'type': 'batsman',
                'average': batting_info['Avg'].values[0],
                'strike_rate': batting_info['Strike Rate'].values[0]
            }
        if player_name in bowling_data['Player'].values:
            bowling_info = bowling_data[bowling_data['Player'] == player_name]
            predictions[player_name].update({
                'type': 'bowler',
                'average': bowling_info['Avg'].values[0],
                'economy': bowling_info['Econ'].values[0]
            })
    print(f"Predictions: {predictions}")
    return predictions

# GUI Development
def show_team():
    team = entry_team.get()
    opponent_team = entry_opponent_team.get()
    stadium_condition = entry_stadium_condition.get()
    
    print(f"Input: Team={team}, Opponent Team={opponent_team}, Stadium Condition={stadium_condition}")

    if team.lower() == 'india':
        team_batting_data = ind_batting_vs_aus
        team_bowling_data = ind_bowling_vs_aus
    elif team.lower() == 'australia':
        team_batting_data = aus_batting_vs_ind
        team_bowling_data = aus_bowling_vs_ind
    else:
        messagebox.showerror('Error', 'Invalid team name. Please enter "India" or "Australia".')
        return

    if opponent_team.lower() == 'india':
        opponent_batting_data = ind_batting_vs_aus
        opponent_bowling_data = ind_bowling_vs_aus
    elif opponent_team.lower() == 'australia':
        opponent_batting_data = aus_batting_vs_ind
        opponent_bowling_data = aus_bowling_vs_ind
    else:
        messagebox.showerror('Error', 'Invalid opponent team name. Please enter "India" or "Australia".')
        return

    selected_team = select_best_11(team_batting_data, team_bowling_data, stadium_condition)
    if selected_team.empty:
        messagebox.showerror('Error', 'Failed to generate team. Please check the input data.')
        return
    
    print("Selected Team DataFrame:")
    print(selected_team)

    predictions = predict_performance(selected_team, team_batting_data, team_bowling_data)
    print("Predictions:")
    print(predictions)
    
    result_text = f"Selected Team for {team}:\n"
    for player, stats in predictions.items():
        result_text += f"{player} - {stats.get('type', '')}\n"
        if 'average' in stats:
            result_text += f"  Average: {stats['average']}\n"
        if 'strike_rate' in stats:
            result_text += f"  Strike Rate: {stats['strike_rate']}\n"
        if 'economy' in stats:
            result_text += f"  Economy: {stats['economy']}\n"
    
    messagebox.showinfo('Predicted Team', result_text)

# Create GUI window
window = tk.Tk()
window.title('Cricket Team Predictor')

# Add input fields
tk.Label(window, text='Team').pack()
entry_team = tk.Entry(window)
entry_team.pack()

tk.Label(window, text='Opponent Team').pack()
entry_opponent_team = tk.Entry(window)
entry_opponent_team.pack()

tk.Label(window, text='Stadium Condition').pack()
entry_stadium_condition = tk.Entry(window)
entry_stadium_condition.pack()

# Add predict button
tk.Button(window, text='Predict Team', command=show_team).pack()

# Run the GUI loop
window.mainloop()


In [ ]:
import pandas as pd
import tkinter as tk
from tkinter import messagebox

# Load the CSV files
ind_batting_vs_aus = pd.read_csv('E:\\Major project\\ind_batting_against_aus.csv')
ind_bowling_vs_aus = pd.read_csv('E:\\Major project\\ind_bowling_against_aus.csv')
aus_batting_vs_ind = pd.read_csv('E:\\Major project\\aus_batting_against_ind.csv')
aus_bowling_vs_ind = pd.read_csv('E:\\Major project\\aus_bowling_against_ind.csv')
stadium_conditions = pd.read_csv('E:\\Major project\\stadium_conditions.csv')

# Handle missing values
ind_batting_vs_aus.fillna(method='ffill', inplace=True)
ind_bowling_vs_aus.fillna(method='ffill', inplace=True)
aus_batting_vs_ind.fillna(method='ffill', inplace=True)
aus_bowling_vs_ind.fillna(method='ffill', inplace=True)
stadium_conditions.fillna(method='ffill', inplace=True)

# Print column names for debugging
print("Indian Batting vs Australia Bowlers Columns:", ind_batting_vs_aus.columns)
print("Indian Bowling vs Australia Batsmen Columns:", ind_bowling_vs_aus.columns)
print("Australian Batting vs India Bowlers Columns:", aus_batting_vs_ind.columns)
print("Australian Bowling vs India Batsmen Columns:", aus_bowling_vs_ind.columns)
print("Stadium Conditions Columns:", stadium_conditions.columns)

# Ensure the relevant columns are numeric
def convert_columns_to_numeric(df, columns):
    for col in columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        else:
            print(f"Warning: Column '{col}' not found in DataFrame")
    return df

ind_batting_vs_aus = convert_columns_to_numeric(ind_batting_vs_aus, ['Avg', 'Strike Rate', 'Runs'])
ind_bowling_vs_aus = convert_columns_to_numeric(ind_bowling_vs_aus, ['Avg', 'Econ'])
aus_batting_vs_ind = convert_columns_to_numeric(aus_batting_vs_ind, ['Avg', 'Strike Rate', 'Runs'])
aus_bowling_vs_ind = convert_columns_to_numeric(aus_bowling_vs_ind, ['Ave', 'Econ'])

# Convert Span to numeric
def convert_span_to_numeric(df):
    if 'Span' in df.columns:
        df['Span'] = df['Span'].apply(lambda x: x.split('-') if isinstance(x, str) else [0, 0])
        df['Span'] = df['Span'].apply(lambda x: int(x[1]) - int(x[0]))
    return df

ind_batting_vs_aus = convert_span_to_numeric(ind_batting_vs_aus)
ind_bowling_vs_aus = convert_span_to_numeric(ind_bowling_vs_aus)
aus_batting_vs_ind = convert_span_to_numeric(aus_batting_vs_ind)
aus_bowling_vs_ind = convert_span_to_numeric(aus_bowling_vs_ind)

# Fill any remaining NaN values with 0
ind_batting_vs_aus.fillna(0, inplace=True)
ind_bowling_vs_aus.fillna(0, inplace=True)
aus_batting_vs_ind.fillna(0, inplace=True)
aus_bowling_vs_ind.fillna(0, inplace=True)

# Function to select the best 11 players
def select_best_11(team_batting, team_bowling, stadium_condition):
    stadium_info = stadium_conditions[stadium_conditions['Stadium Name'] == stadium_condition]
    
    if stadium_info.empty:
        print("Stadium not found in stadium conditions dataset.")
        return pd.DataFrame()
    
    pitch_type = stadium_info.iloc[0]['Pitch Type']
    is_batsman_friendly = stadium_info.iloc[0]['Batsman Friendly (1/0)']
    is_bowler_friendly = stadium_info.iloc[0]['Bowler Friendly (1/0)']
    
    # Print stadium info for debugging
    print(f"Stadium Info: {stadium_info}")
    
    # Add a priority score for batsmen based on average, strike rate, runs, and span
    team_batting['Priority'] = (team_batting['Avg'] * 0.4 + 
                                team_batting['Strike Rate'] * 0.3 + 
                                team_batting['Runs'] * 0.2 + 
                                team_batting['Span'] * 0.1)
    
    # Handle the different column name for average in team_bowling
    avg_column = 'Avg' if 'Avg' in team_bowling.columns else 'Ave'
    
    # Add a priority score for bowlers based on average, economy, and span
    team_bowling['Priority'] = (team_bowling[avg_column] * 0.4 + 
                                team_bowling['Econ'] * 0.4 + 
                                team_bowling['Span'] * 0.2)
    
    # Sort players based on their priority score
    best_batsmen = team_batting.sort_values(by='Priority', ascending=False)
    best_bowlers = team_bowling.sort_values(by='Priority', ascending=False)
    
    # Select best all-rounders
    all_rounders = team_bowling[team_bowling['Player'].isin(team_batting['Player'])]
    best_all_rounders = all_rounders.sort_values(by='Priority', ascending=False)
    
    # Select top 6 batsmen
    top_batsmen = best_batsmen.head(6)
    
    # Select top 4 bowlers
    top_bowlers = best_bowlers.head(4)
    
    # Select top 2 all-rounders
    top_all_rounders = best_all_rounders.head(2)
    
    # Combine the best players ensuring no duplicates
    selected_team = pd.concat([top_batsmen, top_bowlers, top_all_rounders]).drop_duplicates().head(11)
    
    print(f"Selected Team Data: {selected_team}")
    
    return selected_team

# Function to predict performance
def predict_performance(player_data, batting_data, bowling_data):
    predictions = {}
    for _, player in player_data.iterrows():
        player_name = player['Player']
        predictions[player_name] = {}
        
        if player_name in batting_data['Player'].values:
            batting_info = batting_data[batting_data['Player'] == player_name]
            predictions[player_name]['type'] = 'batsman'
            predictions[player_name]['average'] = batting_info['Avg'].values[0] if not pd.isna(batting_info['Avg'].values[0]) else 'N/A'
            predictions[player_name]['strike_rate'] = batting_info['Strike Rate'].values[0] if not pd.isna(batting_info['Strike Rate'].values[0]) else 'N/A'
        
        if player_name in bowling_data['Player'].values:
            bowling_info = bowling_data[bowling_data['Player'] == player_name]
            avg_column = 'Avg' if 'Avg' in bowling_info.columns else 'Ave'
            avg_value = bowling_info[avg_column].values[0] if not pd.isna(bowling_info[avg_column].values[0]) else 'N/A'
            econ_value = bowling_info['Econ'].values[0] if not pd.isna(bowling_info['Econ'].values[0]) else 'N/A'
            
            if 'type' in predictions[player_name]:
                predictions[player_name]['type'] = 'all-rounder'
            else:
                predictions[player_name]['type'] = 'bowler'
            
            predictions[player_name]['average'] = avg_value if avg_value != 0 else 'N/A'
            predictions[player_name]['economy'] = econ_value if econ_value != 0 else 'N/A'
    
    print(f"Predictions: {predictions}")
    return predictions

# Function to show the selected team
def show_team():
    team = entry_team.get()
    opponent_team = entry_opponent_team.get()
    stadium_condition = entry_stadium_condition.get()
    
    if team == 'India' and opponent_team == 'Australia':
        team_batting_data = ind_batting_vs_aus
        team_bowling_data = ind_bowling_vs_aus
    elif team == 'Australia' and opponent_team == 'India':
        team_batting_data = aus_batting_vs_ind
        team_bowling_data = aus_bowling_vs_ind
    else:
        messagebox.showerror("Error", "Invalid team selection.")
        return
    
    stadium_info = stadium_conditions[stadium_conditions['Stadium Name'] == stadium_condition]
    if stadium_info.empty:
        messagebox.showerror("Error", "Invalid stadium condition.")
        return
    
    # Print input info for debugging
    print(f"Input: Team={team}, Opponent Team={opponent_team}, Stadium Condition={stadium_condition}")
    
    selected_team = select_best_11(team_batting_data, team_bowling_data, stadium_condition)
    predictions = predict_performance(selected_team, team_batting_data, team_bowling_data)
    
    result_text = "Selected Team:\n"
    for player, stats in predictions.items():
        result_text += f"{player} ({stats['type']}):\n"
        if 'average' in stats:
            result_text += f"  Average: {stats['average']}\n"
        if 'strike_rate' in stats:
            result_text += f"  Strike Rate: {stats['strike_rate']}\n"
        if 'economy' in stats:
            result_text += f"  Economy: {stats['economy']}\n"
    
    messagebox.showinfo('Selected Team', result_text)

# Create GUI window
window = tk.Tk()
window.title('Cricket Team Predictor')

# Add input fields
tk.Label(window, text='Team').pack()
entry_team = tk.Entry(window)
entry_team.pack()

tk.Label(window, text='Opponent Team').pack()
entry_opponent_team = tk.Entry(window)
entry_opponent_team.pack()

tk.Label(window, text='Stadium Condition').pack()
entry_stadium_condition = tk.Entry(window)
entry_stadium_condition.pack()

# Add predict button
tk.Button(window, text='Predict Team', command=show_team).pack()

# Run the GUI loop
window.mainloop()


In [ ]:
import pandas as pd
import tkinter as tk
from tkinter import messagebox

# Load the CSV files from the specified path
path = r'E:\Major project\\'

ind_batting_vs_aus = pd.read_csv(path + 'ind_batting_against_aus.csv')
ind_bowling_vs_aus = pd.read_csv(path + 'ind_bowling_against_aus.csv')
aus_batting_vs_ind = pd.read_csv(path + 'aus_batting_against_ind.csv')
aus_bowling_vs_ind = pd.read_csv(path + 'aus_bowling_against_ind.csv')

ind_batting_vs_eng = pd.read_csv(path + 'ind_batting_against_eng.csv')
ind_bowling_vs_eng = pd.read_csv(path + 'ind_bowling_against_eng.csv')
eng_batting_vs_ind = pd.read_csv(path + 'eng_batting_against_ind.csv')
eng_bowling_vs_ind = pd.read_csv(path + 'eng_bowling_against_ind.csv')

stadium_conditions = pd.read_csv(path + 'stadium_conditions.csv')

# Handle missing values
ind_batting_vs_aus.fillna(method='ffill', inplace=True)
ind_bowling_vs_aus.fillna(method='ffill', inplace=True)
aus_batting_vs_ind.fillna(method='ffill', inplace=True)
aus_bowling_vs_ind.fillna(method='ffill', inplace=True)

ind_batting_vs_eng.fillna(method='ffill', inplace=True)
ind_bowling_vs_eng.fillna(method='ffill', inplace=True)
eng_batting_vs_ind.fillna(method='ffill', inplace=True)
eng_bowling_vs_ind.fillna(method='ffill', inplace=True)

stadium_conditions.fillna(method='ffill', inplace=True)

# Print column names for debugging
print("Indian Batting vs Australia Bowlers Columns:", ind_batting_vs_aus.columns)
print("Indian Bowling vs Australia Batsmen Columns:", ind_bowling_vs_aus.columns)
print("Australian Batting vs India Bowlers Columns:", aus_batting_vs_ind.columns)
print("Australian Bowling vs India Batsmen Columns:", aus_bowling_vs_ind.columns)

print("Indian Batting vs England Bowlers Columns:", ind_batting_vs_eng.columns)
print("Indian Bowling vs England Batsmen Columns:", ind_bowling_vs_eng.columns)
print("England Batting vs India Bowlers Columns:", eng_batting_vs_ind.columns)
print("England Bowling vs India Batsmen Columns:", eng_bowling_vs_ind.columns)

print("Stadium Conditions Columns:", stadium_conditions.columns)

# Check if the required columns exist and rename them if necessary
def check_and_rename_columns(df, column_mapping):
    for required_col, alternative_cols in column_mapping.items():
        for col in alternative_cols:
            if col in df.columns:
                df.rename(columns={col: required_col}, inplace=True)
    return df

# Define the required and alternative column names
column_mapping = {
    'Avg': ['Avg', 'Ave'],
    'Strike Rate': ['Strike Rate', 'Strike rate', 'SR'],
    'Runs': ['Runs', 'R'],
    'Econ': ['Econ', 'Economy'],
    'Wkts': ['Wkts', 'W']
}

# Check and rename columns
ind_batting_vs_aus = check_and_rename_columns(ind_batting_vs_aus, column_mapping)
ind_bowling_vs_aus = check_and_rename_columns(ind_bowling_vs_aus, column_mapping)
aus_batting_vs_ind = check_and_rename_columns(aus_batting_vs_ind, column_mapping)
aus_bowling_vs_ind = check_and_rename_columns(aus_bowling_vs_ind, column_mapping)

ind_batting_vs_eng = check_and_rename_columns(ind_batting_vs_eng, column_mapping)
ind_bowling_vs_eng = check_and_rename_columns(ind_bowling_vs_eng, column_mapping)
eng_batting_vs_ind = check_and_rename_columns(eng_batting_vs_ind, column_mapping)
eng_bowling_vs_ind = check_and_rename_columns(eng_bowling_vs_ind, column_mapping)

# Convert columns to numeric where necessary
def convert_columns_to_numeric(df, columns):
    for column in columns:
        df[column] = pd.to_numeric(df[column], errors='coerce')
    return df

# Apply the conversion
ind_batting_vs_aus = convert_columns_to_numeric(ind_batting_vs_aus, ['Avg', 'Strike Rate', 'Runs'])
ind_bowling_vs_aus = convert_columns_to_numeric(ind_bowling_vs_aus, ['Avg', 'Econ', 'Wkts'])
aus_batting_vs_ind = convert_columns_to_numeric(aus_batting_vs_ind, ['Avg', 'Strike Rate', 'Runs'])
aus_bowling_vs_ind = convert_columns_to_numeric(aus_bowling_vs_ind, ['Avg', 'Econ', 'Wkts'])

ind_batting_vs_eng = convert_columns_to_numeric(ind_batting_vs_eng, ['Avg', 'Strike Rate', 'Runs'])
ind_bowling_vs_eng = convert_columns_to_numeric(ind_bowling_vs_eng, ['Avg', 'Econ', 'Wkts'])
eng_batting_vs_ind = convert_columns_to_numeric(eng_batting_vs_ind, ['Avg', 'Strike Rate', 'Runs'])
eng_bowling_vs_ind = convert_columns_to_numeric(eng_bowling_vs_ind, ['Avg', 'Econ', 'Wkts'])

# Function to select the best 11 players
def select_best_11(team_batting, team_bowling, stadium_condition):
    stadium_info = stadium_conditions[stadium_conditions['Stadium Name'] == stadium_condition]
    
    if stadium_info.empty:
        print("Stadium not found in stadium conditions dataset.")
        return pd.DataFrame()
    
    pitch_type = stadium_info.iloc[0]['Pitch Type']
    is_batsman_friendly = stadium_info.iloc[0]['Batsman Friendly (1/0)']
    is_bowler_friendly = stadium_info.iloc[0]['Bowler Friendly (1/0)']
    
    # Print stadium info for debugging
    print(f"Stadium Info: {stadium_info}")
    
    # Sort players based on their average and strike rate for batsmen
    team_batting['Priority'] = (team_batting['Avg'] * 0.4 +
                                team_batting['Strike Rate'] * 0.4 +
                                team_batting['Runs'] * 0.2)
    
    # Sort players based on their bowling average and economy for bowlers
    team_bowling['Priority'] = (team_bowling['Avg'] * 0.4 +
                                team_bowling['Econ'] * 0.4 +
                                team_bowling['Wkts'] * 0.2)
    
    best_batsmen = team_batting.sort_values(by=['Priority'], ascending=False)
    best_bowlers = team_bowling.sort_values(by=['Priority'], ascending=True)
    
    # Select the best players
    top_batsmen = best_batsmen.head(6)
    best_bowlers = best_bowlers.head(4)
    
    # Select 2 best all-rounders
    all_rounders = team_bowling[team_bowling['Player'].isin(team_batting['Player'])]
    best_all_rounders = all_rounders.sort_values(by=['Priority'], ascending=True).head(2)
    
    # Combine the best players
    selected_team = pd.concat([top_batsmen, best_bowlers, best_all_rounders]).drop_duplicates().head(11)
    
    print(f"Selected Team Data: {selected_team}")
    
    return selected_team

# Function to predict performance
def predict_performance(player_data, batting_data, bowling_data):
    predictions = {}
    for _, player in player_data.iterrows():
        player_name = player['Player']
        predictions[player_name] = {}
        if player_name in batting_data['Player'].values:
            batting_info = batting_data[batting_data['Player'] == player_name]
            predictions[player_name] = {
                'type': 'batsman',
                'average': batting_info['Avg'].values[0],
                'strike_rate': batting_info['Strike Rate'].values[0]
            }
        if player_name in bowling_data['Player'].values:
            bowling_info = bowling_data[bowling_data['Player'] == player_name]
            predictions[player_name].update({
                'type': 'bowler',
                'average': bowling_info['Avg'].values[0],
                'economy': bowling_info['Econ'].values[0]
            })
    print(f"Predictions: {predictions}")
    return predictions

# GUI Development
def show_team():
    team = entry_team.get()
    opponent_team = entry_opponent_team.get()
    stadium_condition = entry_stadium_condition.get()
    
    print(f"Input: Team={team}, Opponent Team={opponent_team}, Stadium Condition={stadium_condition}")

    if team == 'India' and opponent_team == 'Australia':
        team_batting_data = ind_batting_vs_aus
        team_bowling_data = ind_bowling_vs_aus
    elif team == 'Australia' and opponent_team == 'India':
        team_batting_data = aus_batting_vs_ind
        team_bowling_data = aus_bowling_vs_ind
    elif team == 'India' and opponent_team == 'England':
        team_batting_data = ind_batting_vs_eng
        team_bowling_data = ind_bowling_vs_eng
    elif team == 'England' and opponent_team == 'India':
        team_batting_data = eng_batting_vs_ind
        team_bowling_data = eng_bowling_vs_ind
    else:
        messagebox.showerror('Error', 'Invalid team or opponent team selection')
        return
    
    selected_team = select_best_11(team_batting_data, team_bowling_data, stadium_condition)
    
    if selected_team.empty:
        messagebox.showerror('Error', 'No players found for the selected team and conditions')
        return

    predictions = predict_performance(selected_team, team_batting_data, team_bowling_data)
    
    result_text = ""
    for player in predictions:
        result_text += f"{player}: {predictions[player]}\n"
    
    messagebox.showinfo('Selected Team', result_text)

# Create the main window
window = tk.Tk()
window.title('Team Selection and Performance Prediction')

# Add input fields
tk.Label(window, text='Team').grid(row=0, column=0)
entry_team = tk.Entry(window)
entry_team.grid(row=0, column=1)

tk.Label(window, text='Opponent Team').grid(row=1, column=0)
entry_opponent_team = tk.Entry(window)
entry_opponent_team.grid(row=1, column=1)

tk.Label(window, text='Stadium Condition').grid(row=2, column=0)
entry_stadium_condition = tk.Entry(window)
entry_stadium_condition.grid(row=2, column=1)

# Add button to show team
tk.Button(window, text='Show Team', command=show_team).grid(row=3, column=0, columnspan=2)

# Run the application
window.mainloop()


In [ ]:
import pandas as pd
import tkinter as tk
from tkinter import ttk
import numpy as np

# Load CSV files
path = r'E:\Major project\\'

# Existing teams
ind_batting_vs_aus = pd.read_csv(path + 'ind_batting_against_aus.csv')
ind_bowling_vs_aus = pd.read_csv(path + 'ind_bowling_against_aus.csv')
aus_batting_vs_ind = pd.read_csv(path + 'aus_batting_against_ind.csv')
aus_bowling_vs_ind = pd.read_csv(path + 'aus_bowling_against_ind.csv')

ind_batting_vs_eng = pd.read_csv(path + 'ind_batting_against_eng.csv')
ind_bowling_vs_eng = pd.read_csv(path + 'ind_bowling_against_eng.csv')
eng_batting_vs_ind = pd.read_csv(path + 'eng_batting_against_ind.csv')
eng_bowling_vs_ind = pd.read_csv(path + 'eng_bowling_against_ind.csv')

ind_batting_vs_pak = pd.read_csv(path + 'ind_batting_against_pak.csv')
ind_bowling_vs_pak = pd.read_csv(path + 'ind_bowling_against_pak.csv')
pak_batting_vs_ind = pd.read_csv(path + 'pak_batting_against_ind.csv')
pak_bowling_vs_ind = pd.read_csv(path + 'pak_bowling_against_ind.csv')

# New teams
ind_batting_vs_sa = pd.read_csv(path + 'ind_batting_against_sa.csv')
ind_bowling_vs_sa = pd.read_csv(path + 'ind_bowling_against_sa.csv')
sa_batting_vs_ind = pd.read_csv(path + 'sa_batting _against_ind.csv')
sa_bowling_vs_ind = pd.read_csv(path + 'sa_bowling_against_ind.csv')

ind_batting_vs_sri = pd.read_csv(path + 'ind_batting_against_sri.csv')
ind_bowling_vs_sri = pd.read_csv(path + 'ind_bowling_against_sri.csv')
sri_batting_vs_ind = pd.read_csv(path + 'sri_batting_against_ind.csv')
sri_bowling_vs_ind = pd.read_csv(path + 'sri_bowling_against_ind.csv')

stadium_conditions = pd.read_csv(path + 'stadium_conditions.csv')

# Rename stadium conditions columns
stadium_conditions.rename(columns={'Stadium Name': 'stadium_name'}, inplace=True)

# Renaming dictionaries
batting_rename_dict = {
    'Avg': 'avg',
    'Strike Rate': 'strike_rate',
    'Runs': 'runs',
    'Strike rate': 'strike_rate',
    'Not out': 'Not Out'
}

bowling_rename_dict = {
    'Avg': 'avg',
    'Ave': 'avg',
    'Econ': 'econ'
}

# Rename columns to ensure consistency
def rename_columns(df, rename_dict):
    df.rename(columns=rename_dict, inplace=True)
    return df

# Apply renaming
dataframes = [
    ind_batting_vs_aus, ind_bowling_vs_aus, aus_batting_vs_ind, aus_bowling_vs_ind,
    ind_batting_vs_eng, ind_bowling_vs_eng, eng_batting_vs_ind, eng_bowling_vs_ind,
    ind_batting_vs_pak, ind_bowling_vs_pak, pak_batting_vs_ind, pak_bowling_vs_ind,
    ind_batting_vs_sa, ind_bowling_vs_sa, sa_batting_vs_ind, sa_bowling_vs_ind,
    ind_batting_vs_sri, ind_bowling_vs_sri, sri_batting_vs_ind, sri_bowling_vs_ind
]

for df in dataframes:
    rename_columns(df, batting_rename_dict)
    rename_columns(df, bowling_rename_dict)

# Convert relevant columns to numeric
def convert_columns_to_numeric(df, columns):
    for column in columns:
        if column in df.columns:
            df[column] = pd.to_numeric(df[column], errors='coerce')
        else:
            print(f"Warning: Column '{column}' not found in DataFrame")
    return df

required_batting_columns = ['avg', 'strike_rate', 'runs']
required_bowling_columns = ['avg', 'econ']

# Apply conversion
for df in dataframes:
    convert_columns_to_numeric(df, required_batting_columns)
    convert_columns_to_numeric(df, required_bowling_columns)

# Forward fill missing stadium conditions
stadium_conditions = stadium_conditions.ffill()

# Fill NaN values with relevant random values
def fill_na_with_random(df, columns):
    for column in columns:
        if column in df.columns:
            mean_value = df[column].mean()
            std_value = df[column].std()
            random_values = np.random.normal(mean_value, std_value, df[column].isna().sum())
            df.loc[df[column].isna(), column] = random_values
        else:
            print(f"Warning: Column '{column}' not found in DataFrame")
    return df

# Apply fill_na_with_random
for df in dataframes:
    fill_na_with_random(df, required_batting_columns)
    fill_na_with_random(df, required_bowling_columns)

# Select best 11 players with a balance of 6 batsmen and 5 bowlers
def select_best_11(batting_df, bowling_df):
    # Ensure required columns exist
    batting_df = ensure_columns_exist(batting_df, ['avg', 'strike_rate'])
    bowling_df = ensure_columns_exist(bowling_df, ['avg', 'econ'])

    # Select top 6 batsmen based on average
    best_batsmen = batting_df.sort_values(by='avg', ascending=False).head(6)
    # Select top 5 bowlers based on average
    best_bowlers = bowling_df.sort_values(by='avg', ascending=True).head(5)

    # Combine all selected players
    best_11 = pd.concat([best_batsmen, best_bowlers], ignore_index=True)

    # Ensure exactly 11 players
    if len(best_11) < 11:
        # If less than 11 players are found, add more from remaining players
        remaining_players = pd.concat([batting_df, bowling_df]).drop_duplicates()
        additional_players = remaining_players[~remaining_players['Player'].isin(best_11['Player'])].head(11 - len(best_11))
        best_11 = pd.concat([best_11, additional_players], ignore_index=True)

    return best_11

# Ensure columns exist in the DataFrame
def ensure_columns_exist(df, columns):
    for column in columns:
        if column not in df.columns:
            df[column] = np.nan
    return df

# GUI Implementation
class CricketTeamSelector:
    def __init__(self, root):
        self.root = root
        self.root.title("Cricket Team Selector")
        self.root.geometry("600x400")

        self.team_label = tk.Label(root, text="Select Team:")
        self.team_label.pack()

        self.team_combobox = ttk.Combobox(root, values=["India", "Australia", "England", "Pakistan", "South Africa", "Sri Lanka"])
        self.team_combobox.pack()

        self.opponent_label = tk.Label(root, text="Select Opponent:")
        self.opponent_label.pack()

        self.opponent_combobox = ttk.Combobox(root, values=["India", "Australia", "England", "Pakistan", "South Africa", "Sri Lanka"])
        self.opponent_combobox.pack()

        self.stadium_label = tk.Label(root, text="Select Stadium:")
        self.stadium_label.pack()

        self.stadium_combobox = ttk.Combobox(root, values=stadium_conditions['stadium_name'].unique().tolist())
        self.stadium_combobox.pack()

        self.show_button = tk.Button(root, text="Show Best Team", command=self.show_team)
        self.show_button.pack()

        self.team_output = tk.Text(root, height=15, width=70)
        self.team_output.pack()

    def show_team(self):
        selected_team = self.team_combobox.get()
        opponent_team = self.opponent_combobox.get()
        stadium_name = self.stadium_combobox.get()

        if stadium_name in stadium_conditions['stadium_name'].values:
            stadium_info = stadium_conditions[stadium_conditions['stadium_name'] == stadium_name].iloc[0]
        else:
            self.team_output.delete(1.0, tk.END)
            self.team_output.insert(tk.END, "Invalid stadium selection.")
            return

        # Determine which DataFrame to use based on selected teams
        if selected_team == "India" and opponent_team == "Australia":
            team_batting_data = ind_batting_vs_aus
            team_bowling_data = ind_bowling_vs_aus
        elif selected_team == "Australia" and opponent_team == "India":
            team_batting_data = aus_batting_vs_ind
            team_bowling_data = aus_bowling_vs_ind
        elif selected_team == "India" and opponent_team == "England":
            team_batting_data = ind_batting_vs_eng
            team_bowling_data = ind_bowling_vs_eng
        elif selected_team == "England" and opponent_team == "India":
            team_batting_data = eng_batting_vs_ind
            team_bowling_data = eng_bowling_vs_ind
        elif selected_team == "India" and opponent_team == "Pakistan":
            team_batting_data = ind_batting_vs_pak
            team_bowling_data = ind_bowling_vs_pak
        elif selected_team == "Pakistan" and opponent_team == "India":
            team_batting_data = pak_batting_vs_ind
            team_bowling_data = pak_bowling_vs_ind
        elif selected_team == "India" and opponent_team == "South Africa":
            team_batting_data = ind_batting_vs_sa
            team_bowling_data = ind_bowling_vs_sa
        elif selected_team == "South Africa" and opponent_team == "India":
            team_batting_data = sa_batting_vs_ind
            team_bowling_data = sa_bowling_vs_ind
        elif selected_team == "India" and opponent_team == "Sri Lanka":
            team_batting_data = ind_batting_vs_sri
            team_bowling_data = ind_bowling_vs_sri
        elif selected_team == "Sri Lanka" and opponent_team == "India":
            team_batting_data = sri_batting_vs_ind
            team_bowling_data = sri_bowling_vs_ind
        else:
            self.team_output.delete(1.0, tk.END)
            self.team_output.insert(tk.END, "Invalid team or opponent selection.")
            return

        best_11 = select_best_11(team_batting_data, team_bowling_data)

        self.team_output.delete(1.0, tk.END)

        for _, row in best_11.iterrows():
            if 'strike_rate' in row and 'avg' in row:
                if row['Player'] in team_batting_data['Player'].values:
                    self.team_output.insert(tk.END, f"Player: {row['Player']} - Strike Rate: {row['strike_rate']}, Batting Avg: {row['avg']}\n")
                elif row['Player'] in team_bowling_data['Player'].values:
                    self.team_output.insert(tk.END, f"Player: {row['Player']} - Bowling Avg: {row['avg']}, Economy: {row.get('econ', 'N/A')}\n")

root = tk.Tk()
app = CricketTeamSelector(root)
root.mainloop()


till here Ind vs 5 teams completed

In [ ]:
import pandas as pd
import tkinter as tk
from tkinter import ttk
import numpy as np

# Load CSV files
path = r'E:\Major project\\'

# Existing teams
ind_batting_vs_aus = pd.read_csv(path + 'ind_batting_against_aus.csv')
ind_bowling_vs_aus = pd.read_csv(path + 'ind_bowling_against_aus.csv')
aus_batting_vs_ind = pd.read_csv(path + 'aus_batting_against_ind.csv')
aus_bowling_vs_ind = pd.read_csv(path + 'aus_bowling_against_ind.csv')

ind_batting_vs_eng = pd.read_csv(path + 'ind_batting_against_eng.csv')
ind_bowling_vs_eng = pd.read_csv(path + 'ind_bowling_against_eng.csv')
eng_batting_vs_ind = pd.read_csv(path + 'eng_batting_against_ind.csv')
eng_bowling_vs_ind = pd.read_csv(path + 'eng_bowling_against_ind.csv')

ind_batting_vs_pak = pd.read_csv(path + 'ind_batting_against_pak.csv')
ind_bowling_vs_pak = pd.read_csv(path + 'ind_bowling_against_pak.csv')
pak_batting_vs_ind = pd.read_csv(path + 'pak_batting_against_ind.csv')
pak_bowling_vs_ind = pd.read_csv(path + 'pak_bowling_against_ind.csv')

# New teams
ind_batting_vs_sa = pd.read_csv(path + 'ind_batting_against_sa.csv')
ind_bowling_vs_sa = pd.read_csv(path + 'ind_bowling_against_sa.csv')
sa_batting_vs_ind = pd.read_csv(path + 'sa_batting_against_ind.csv')
sa_bowling_vs_ind = pd.read_csv(path + 'sa_bowling_against_ind.csv')

ind_batting_vs_sri = pd.read_csv(path + 'ind_batting_against_sri.csv')
ind_bowling_vs_sri = pd.read_csv(path + 'ind_bowling_against_sri.csv')
sri_batting_vs_ind = pd.read_csv(path + 'sri_batting_against_ind.csv')
sri_bowling_vs_ind = pd.read_csv(path + 'sri_bowling_against_ind.csv')

stadium_conditions = pd.read_csv(path + 'stadium_conditions.csv')

# Rename stadium conditions columns
stadium_conditions.rename(columns={'Stadium Name': 'stadium_name'}, inplace=True)

# Check if 'Condition' column exists, add if missing
if 'Condition' not in stadium_conditions.columns:
    print("Warning: Column 'Condition' is missing from stadium_conditions DataFrame. Adding default values.")
    stadium_conditions['Condition'] = 'Neutral'  # Default value

# Renaming dictionaries
batting_rename_dict = {
    'Avg': 'avg',
    'Strike Rate': 'strike_rate',
    'Runs': 'runs',
    'Strike rate': 'strike_rate',
    'Not out': 'Not Out'
}

bowling_rename_dict = {
    'Avg': 'avg',
    'Ave': 'avg',
    'Econ': 'econ'
}

# Rename columns to ensure consistency
def rename_columns(df, rename_dict):
    df.rename(columns=rename_dict, inplace=True)
    return df

# Apply renaming
dataframes = [
    ind_batting_vs_aus, ind_bowling_vs_aus, aus_batting_vs_ind, aus_bowling_vs_ind,
    ind_batting_vs_eng, ind_bowling_vs_eng, eng_batting_vs_ind, eng_bowling_vs_ind,
    ind_batting_vs_pak, ind_bowling_vs_pak, pak_batting_vs_ind, pak_bowling_vs_ind,
    ind_batting_vs_sa, ind_bowling_vs_sa, sa_batting_vs_ind, sa_bowling_vs_ind,
    ind_batting_vs_sri, ind_bowling_vs_sri, sri_batting_vs_ind, sri_bowling_vs_ind
]

for df in dataframes:
    rename_columns(df, batting_rename_dict)
    rename_columns(df, bowling_rename_dict)

# Convert relevant columns to numeric
def convert_columns_to_numeric(df, columns):
    for column in columns:
        if column in df.columns:
            df[column] = pd.to_numeric(df[column], errors='coerce')
        else:
            print(f"Warning: Column '{column}' not found in DataFrame")
    return df

required_batting_columns = ['avg', 'strike_rate', 'runs']
required_bowling_columns = ['avg', 'econ']

# Apply conversion
for df in dataframes:
    convert_columns_to_numeric(df, required_batting_columns)
    convert_columns_to_numeric(df, required_bowling_columns)

# Fill NaN values with relevant random values
def fill_na_with_random(df, columns):
    for column in columns:
        if column in df.columns:
            mean_value = df[column].mean()
            std_value = df[column].std()
            random_values = np.random.normal(mean_value, std_value, df[column].isna().sum())
            df.loc[df[column].isna(), column] = random_values
        else:
            print(f"Warning: Column '{column}' not found in DataFrame")
    return df

# Apply fill_na_with_random
for df in dataframes:
    fill_na_with_random(df, required_batting_columns)
    fill_na_with_random(df, required_bowling_columns)

# Ensure columns exist in the DataFrame
def ensure_columns_exist(df, columns):
    for column in columns:
        if column not in df.columns:
            if column == 'econ':
                df[column] = np.random.normal(4.0, 0.5, len(df))  # Assuming an average economy of 4.0
            else:
                df[column] = np.nan
    return df

# Apply ensure_columns_exist
for df in dataframes:
    ensure_columns_exist(df, required_batting_columns)
    ensure_columns_exist(df, required_bowling_columns)

# Select best 11 players with a balance of 6 batsmen and 5 bowlers, adjusted for stadium conditions
def select_best_11(batting_df, bowling_df, stadium_info):
    # Ensure required columns exist
    batting_df = ensure_columns_exist(batting_df, ['avg', 'strike_rate'])
    bowling_df = ensure_columns_exist(bowling_df, ['avg', 'econ'])

    # Determine stadium preference
    if stadium_info['Condition'].values[0] == 'Batting':
        batting_weight = 1.5
        bowling_weight = 0.75
    elif stadium_info['Condition'].values[0] == 'Bowling':
        batting_weight = 0.75
        bowling_weight = 1.5
    else:
        batting_weight = 1.0
        bowling_weight = 1.0

    # Adjust player metrics based on stadium conditions and add randomness
    batting_df['adjusted_avg'] = batting_df['avg'] * batting_weight * np.random.uniform(0.95, 1.05)
    bowling_df['adjusted_avg'] = bowling_df['avg'] * bowling_weight * np.random.uniform(0.95, 1.05)

    # Select top 6 batsmen based on adjusted average
    best_batsmen = batting_df.sort_values(by='adjusted_avg', ascending=False).head(6)
    # Select top 5 bowlers based on adjusted average
    best_bowlers = bowling_df.sort_values(by='adjusted_avg', ascending=True).head(5)

    # Combine all selected players
    best_11 = pd.concat([best_batsmen, best_bowlers], ignore_index=True)

    # Ensure exactly 11 players
    if len(best_11) < 11:
        # If less than 11 players are found, add more from remaining players
        remaining_players = pd.concat([batting_df, bowling_df]).drop_duplicates()
        additional_players = remaining_players[~remaining_players['Player'].isin(best_11['Player'])].head(11 - len(best_11))
        best_11 = pd.concat([best_11, additional_players], ignore_index=True)

    return best_11

# GUI Implementation
class CricketTeamSelector:
    def __init__(self, root):
        self.root = root
        self.root.title("Cricket Team Selector")
        self.root.geometry("600x400")

        self.team_label = tk.Label(root, text="Select Team:")
        self.team_label.pack()

        self.team_combobox = ttk.Combobox(root, values=["India", "Australia", "England", "Pakistan", "South Africa", "Sri Lanka"])
        self.team_combobox.pack()

        self.opponent_label = tk.Label(root, text="Select Opponent:")
        self.opponent_label.pack()

        self.opponent_combobox = ttk.Combobox(root, values=["India", "Australia", "England", "Pakistan", "South Africa", "Sri Lanka"])
        self.opponent_combobox.pack()

        self.stadium_label = tk.Label(root, text="Select Stadium:")
        self.stadium_label.pack()

        self.stadium_combobox = ttk.Combobox(root, values=stadium_conditions['stadium_name'].tolist())
        self.stadium_combobox.pack()

        self.submit_button = tk.Button(root, text="Show Best 11", command=self.show_team)
        self.submit_button.pack()

        self.team_output = tk.Text(root, height=15, width=50)
        self.team_output.pack()

    def show_team(self):
        selected_team = self.team_combobox.get()
        opponent_team = self.opponent_combobox.get()
        stadium_name = self.stadium_combobox.get()

        if stadium_name in stadium_conditions['stadium_name'].values:
            stadium_info = stadium_conditions[stadium_conditions['stadium_name'] == stadium_name]
        else:
            self.team_output.delete(1.0, tk.END)
            self.team_output.insert(tk.END, "Invalid stadium selection.")
            return

        # Determine which DataFrame to use based on selected teams
        if selected_team == "India" and opponent_team == "Australia":
            team_batting_data = ind_batting_vs_aus
            team_bowling_data = ind_bowling_vs_aus
        elif selected_team == "Australia" and opponent_team == "India":
            team_batting_data = aus_batting_vs_ind
            team_bowling_data = aus_bowling_vs_ind
        elif selected_team == "India" and opponent_team == "England":
            team_batting_data = ind_batting_vs_eng
            team_bowling_data = ind_bowling_vs_eng
        elif selected_team == "England" and opponent_team == "India":
            team_batting_data = eng_batting_vs_ind
            team_bowling_data = eng_bowling_vs_ind
        elif selected_team == "India" and opponent_team == "Pakistan":
            team_batting_data = ind_batting_vs_pak
            team_bowling_data = ind_bowling_vs_pak
        elif selected_team == "Pakistan" and opponent_team == "India":
            team_batting_data = pak_batting_vs_ind
            team_bowling_data = pak_bowling_vs_ind
        elif selected_team == "India" and opponent_team == "South Africa":
            team_batting_data = ind_batting_vs_sa
            team_bowling_data = ind_bowling_vs_sa
        elif selected_team == "South Africa" and opponent_team == "India":
            team_batting_data = sa_batting_vs_ind
            team_bowling_data = sa_bowling_vs_ind
        elif selected_team == "India" and opponent_team == "Sri Lanka":
            team_batting_data = ind_batting_vs_sri
            team_bowling_data = ind_bowling_vs_sri
        elif selected_team == "Sri Lanka" and opponent_team == "India":
            team_batting_data = sri_batting_vs_ind
            team_bowling_data = sri_bowling_vs_ind
        else:
            self.team_output.delete(1.0, tk.END)
            self.team_output.insert(tk.END, "Invalid team or opponent selection.")
            return

        best_11 = select_best_11(team_batting_data, team_bowling_data, stadium_info)

        self.team_output.delete(1.0, tk.END)

        for _, row in best_11.iterrows():
            if 'strike_rate' in row and 'avg' in row:
                if row['Player'] in team_batting_data['Player'].values:
                    self.team_output.insert(tk.END, f"Player: {row['Player']} - Strike Rate: {row['strike_rate']}, Batting Avg: {row['avg']}\n")
                elif row['Player'] in team_bowling_data['Player'].values:
                    self.team_output.insert(tk.END, f"Player: {row['Player']} - Bowling Avg: {row['avg']}, Economy: {row.get('econ', 'N/A')}\n")

root = tk.Tk()
app = CricketTeamSelector(root)
root.mainloop()


no change in op it does not change the team when i change the stadium name and for some bowler it is showing batting average rather than economy

In [ ]:
import pandas as pd
import tkinter as tk
from tkinter import ttk
import numpy as np

# Load CSV files
path = r'E:\Major project\\'

# Existing teams
ind_batting_vs_aus = pd.read_csv(path + 'ind_batting_against_aus.csv')
ind_bowling_vs_aus = pd.read_csv(path + 'ind_bowling_against_aus.csv')
aus_batting_vs_ind = pd.read_csv(path + 'aus_batting_against_ind.csv')
aus_bowling_vs_ind = pd.read_csv(path + 'aus_bowling_against_ind.csv')

ind_batting_vs_eng = pd.read_csv(path + 'ind_batting_against_eng.csv')
ind_bowling_vs_eng = pd.read_csv(path + 'ind_bowling_against_eng.csv')
eng_batting_vs_ind = pd.read_csv(path + 'eng_batting_against_ind.csv')
eng_bowling_vs_ind = pd.read_csv(path + 'eng_bowling_against_ind.csv')

ind_batting_vs_pak = pd.read_csv(path + 'ind_batting_against_pak.csv')
ind_bowling_vs_pak = pd.read_csv(path + 'ind_bowling_against_pak.csv')
pak_batting_vs_ind = pd.read_csv(path + 'pak_batting_against_ind.csv')
pak_bowling_vs_ind = pd.read_csv(path + 'pak_bowling_against_ind.csv')

# New teams
ind_batting_vs_sa = pd.read_csv(path + 'ind_batting_against_sa.csv')
ind_bowling_vs_sa = pd.read_csv(path + 'ind_bowling_against_sa.csv')
sa_batting_vs_ind = pd.read_csv(path + 'sa_batting_against_ind.csv')
sa_bowling_vs_ind = pd.read_csv(path + 'sa_bowling_against_ind.csv')

ind_batting_vs_sri = pd.read_csv(path + 'ind_batting_against_sri.csv')
ind_bowling_vs_sri = pd.read_csv(path + 'ind_bowling_against_sri.csv')
sri_batting_vs_ind = pd.read_csv(path + 'sri_batting_against_ind.csv')
sri_bowling_vs_ind = pd.read_csv(path + 'sri_bowling_against_ind.csv')

stadium_conditions = pd.read_csv(path + 'stadium_conditions.csv')

# Rename stadium conditions columns
stadium_conditions.rename(columns={'Stadium Name': 'stadium_name'}, inplace=True)

# Check if 'Condition' column exists, add if missing
if 'Condition' not in stadium_conditions.columns:
    print("Warning: Column 'Condition' is missing from stadium_conditions DataFrame. Adding default values.")
    stadium_conditions['Condition'] = 'Neutral'  # Default value

# Renaming dictionaries
batting_rename_dict = {
    'Avg': 'avg',
    'Strike Rate': 'strike_rate',
    'Runs': 'runs',
    'Strike rate': 'strike_rate',
    'Not out': 'Not Out'
}

bowling_rename_dict = {
    'Avg': 'avg',
    'Ave': 'avg',
    'Econ': 'econ'
}

# Rename columns to ensure consistency
def rename_columns(df, rename_dict):
    df.rename(columns=rename_dict, inplace=True)
    return df

# Apply renaming
dataframes = [
    ind_batting_vs_aus, ind_bowling_vs_aus, aus_batting_vs_ind, aus_bowling_vs_ind,
    ind_batting_vs_eng, ind_bowling_vs_eng, eng_batting_vs_ind, eng_bowling_vs_ind,
    ind_batting_vs_pak, ind_bowling_vs_pak, pak_batting_vs_ind, pak_bowling_vs_ind,
    ind_batting_vs_sa, ind_bowling_vs_sa, sa_batting_vs_ind, sa_bowling_vs_ind,
    ind_batting_vs_sri, ind_bowling_vs_sri, sri_batting_vs_ind, sri_bowling_vs_ind
]

for df in dataframes:
    rename_columns(df, batting_rename_dict)
    rename_columns(df, bowling_rename_dict)

# Convert relevant columns to numeric
def convert_columns_to_numeric(df, columns):
    for column in columns:
        if column in df.columns:
            df[column] = pd.to_numeric(df[column], errors='coerce')
        else:
            print(f"Warning: Column '{column}' not found in DataFrame")
    return df

required_batting_columns = ['avg', 'strike_rate', 'runs']
required_bowling_columns = ['avg', 'econ']

# Apply conversion
for df in dataframes:
    convert_columns_to_numeric(df, required_batting_columns)
    convert_columns_to_numeric(df, required_bowling_columns)

# Fill NaN values with relevant random values
def fill_na_with_random(df, columns):
    for column in columns:
        if column in df.columns:
            mean_value = df[column].mean()
            std_value = df[column].std()
            random_values = np.random.normal(mean_value, std_value, df[column].isna().sum())
            df.loc[df[column].isna(), column] = random_values
        else:
            print(f"Warning: Column '{column}' not found in DataFrame")
    return df

# Apply fill_na_with_random
for df in dataframes:
    fill_na_with_random(df, required_batting_columns)
    fill_na_with_random(df, required_bowling_columns)

# Ensure columns exist in the DataFrame
def ensure_columns_exist(df, columns):
    for column in columns:
        if column not in df.columns:
            if column == 'econ':
                df[column] = np.random.normal(4.0, 0.5, len(df))  # Assuming an average economy of 4.0
            else:
                df[column] = np.nan
    return df

# Apply ensure_columns_exist
for df in dataframes:
    ensure_columns_exist(df, required_batting_columns)
    ensure_columns_exist(df, required_bowling_columns)

# Select best 11 players with a balance of 6 batsmen and 5 bowlers, adjusted for stadium conditions
def select_best_11(batting_df, bowling_df, stadium_info):
    # Ensure required columns exist
    batting_df = ensure_columns_exist(batting_df, ['avg', 'strike_rate'])
    bowling_df = ensure_columns_exist(bowling_df, ['avg', 'econ'])

    # Determine stadium preference
    if stadium_info['Condition'].values[0] == 'Batting':
        batting_weight = 1.5
        bowling_weight = 0.75
    elif stadium_info['Condition'].values[0] == 'Bowling':
        batting_weight = 0.75
        bowling_weight = 1.5
    else:
        batting_weight = 1.0
        bowling_weight = 1.0

    # Adjust player metrics based on stadium conditions and add randomness
    batting_df['adjusted_avg'] = batting_df['avg'] * batting_weight * np.random.uniform(0.9, 1.1, len(batting_df))
    bowling_df['adjusted_avg'] = bowling_df['avg'] * bowling_weight * np.random.uniform(0.9, 1.1, len(bowling_df))

    # Select top 6 batsmen based on adjusted average
    best_batsmen = batting_df.sort_values(by='adjusted_avg', ascending=False).head(6)
    # Select top 5 bowlers based on adjusted average
    best_bowlers = bowling_df.sort_values(by='adjusted_avg', ascending=True).head(5)

    # Combine all selected players
    best_11 = pd.concat([best_batsmen, best_bowlers], ignore_index=True)

    # Ensure exactly 11 players
    if len(best_11) < 11:
        # If less than 11 players are found, add more from remaining players
        remaining_players = pd.concat([batting_df, bowling_df]).drop_duplicates()
        additional_players = remaining_players[~remaining_players['Player'].isin(best_11['Player'])].head(11 - len(best_11))
        best_11 = pd.concat([best_11, additional_players], ignore_index=True)

    return best_11

# GUI Implementation
class CricketTeamSelector:
    def __init__(self, root):
        self.root = root
        self.root.title("Cricket Team Selector")
        self.root.geometry("600x400")

        self.team_label = tk.Label(root, text="Select Team:")
        self.team_label.pack()

        self.team_combobox = ttk.Combobox(root, values=["India", "Australia", "England", "Pakistan", "South Africa", "Sri Lanka"])
        self.team_combobox.pack()

        self.opponent_label = tk.Label(root, text="Select Opponent:")
        self.opponent_label.pack()

        self.opponent_combobox = ttk.Combobox(root, values=["India", "Australia", "England", "Pakistan", "South Africa", "Sri Lanka"])
        self.opponent_combobox.pack()

        self.stadium_label = tk.Label(root, text="Select Stadium:")
        self.stadium_label.pack()

        self.stadium_combobox = ttk.Combobox(root, values=stadium_conditions['stadium_name'].tolist())
        self.stadium_combobox.pack()

        self.submit_button = tk.Button(root, text="Show Best 11", command=self.show_team)
        self.submit_button.pack()

        self.team_output = tk.Text(root, height=15, width=50)
        self.team_output.pack()

    def show_team(self):
        selected_team = self.team_combobox.get()
        opponent_team = self.opponent_combobox.get()
        stadium_name = self.stadium_combobox.get()

        if stadium_name in stadium_conditions['stadium_name'].values:
            stadium_info = stadium_conditions[stadium_conditions['stadium_name'] == stadium_name]
        else:
            self.team_output.delete(1.0, tk.END)
            self.team_output.insert(tk.END, "Invalid stadium selection.")
            return

        # Determine which DataFrame to use based on selected teams
        if selected_team == "India" and opponent_team == "Australia":
            team_batting_data = ind_batting_vs_aus
            team_bowling_data = ind_bowling_vs_aus
        elif selected_team == "Australia" and opponent_team == "India":
            team_batting_data = aus_batting_vs_ind
            team_bowling_data = aus_bowling_vs_ind
        elif selected_team == "India" and opponent_team == "England":
            team_batting_data = ind_batting_vs_eng
            team_bowling_data = ind_bowling_vs_eng
        elif selected_team == "England" and opponent_team == "India":
            team_batting_data = eng_batting_vs_ind
            team_bowling_data = eng_bowling_vs_ind
        elif selected_team == "India" and opponent_team == "Pakistan":
            team_batting_data = ind_batting_vs_pak
            team_bowling_data = ind_bowling_vs_pak
        elif selected_team == "Pakistan" and opponent_team == "India":
            team_batting_data = pak_batting_vs_ind
            team_bowling_data = pak_bowling_vs_ind
        elif selected_team == "India" and opponent_team == "South Africa":
            team_batting_data = ind_batting_vs_sa
            team_bowling_data = ind_bowling_vs_sa
        elif selected_team == "South Africa" and opponent_team == "India":
            team_batting_data = sa_batting_vs_ind
            team_bowling_data = sa_bowling_vs_ind
        elif selected_team == "India" and opponent_team == "Sri Lanka":
            team_batting_data = ind_batting_vs_sri
            team_bowling_data = ind_bowling_vs_sri
        elif selected_team == "Sri Lanka" and opponent_team == "India":
            team_batting_data = sri_batting_vs_ind
            team_bowling_data = sri_bowling_vs_ind
        else:
            self.team_output.delete(1.0, tk.END)
            self.team_output.insert(tk.END, "Invalid team or opponent selection.")
            return

        best_11 = select_best_11(team_batting_data, team_bowling_data, stadium_info)

        self.team_output.delete(1.0, tk.END)

        for _, row in best_11.iterrows():
            player_type = "All-Rounder"
            if row['Player'] in team_batting_data['Player'].values and row['Player'] not in team_bowling_data['Player'].values:
                player_type = "Batsman"
            elif row['Player'] in team_bowling_data['Player'].values and row['Player'] not in team_batting_data['Player'].values:
                player_type = "Bowler"

            player_info = f"Player: {row['Player']} - Type: {player_type}"
            if player_type == "Batsman":
                player_info += f", Strike Rate: {row['strike_rate']:.2f}, Batting Avg: {row['avg']:.2f}"
            elif player_type == "Bowler":
                player_info += f", Bowling Avg: {row['avg']:.2f}, Economy: {row['econ']:.2f}"
            else:  # All-Rounder
                if 'strike_rate' in row and not pd.isna(row['strike_rate']):
                    player_info += f", Strike Rate: {row['strike_rate']:.2f}"
                if 'avg' in row and not pd.isna(row['avg']):
                    player_info += f", Batting Avg: {row['avg']:.2f}"
                if 'avg' in row and not pd.isna(row['avg']):
                    player_info += f", Bowling Avg: {row['avg']:.2f}"
                if 'econ' in row and not pd.isna(row['econ']):
                    player_info += f", Economy: {row['econ']:.2f}"

            self.team_output.insert(tk.END, f"{player_info}\n")

root = tk.Tk()
app = CricketTeamSelector(root)
root.mainloop()


now the output is working properly ! 

In [ ]:
import pandas as pd
import tkinter as tk
from tkinter import ttk
import numpy as np

# Load CSV files
path = r'E:\Major project\\'

# Existing teams
ind_batting_vs_aus = pd.read_csv(path + 'ind_batting_against_aus.csv')
ind_bowling_vs_aus = pd.read_csv(path + 'ind_bowling_against_aus.csv')
aus_batting_vs_ind = pd.read_csv(path + 'aus_batting_against_ind.csv')
aus_bowling_vs_ind = pd.read_csv(path + 'aus_bowling_against_ind.csv')

ind_batting_vs_eng = pd.read_csv(path + 'ind_batting_against_eng.csv')
ind_bowling_vs_eng = pd.read_csv(path + 'ind_bowling_against_eng.csv')
eng_batting_vs_ind = pd.read_csv(path + 'eng_batting_against_ind.csv')
eng_bowling_vs_ind = pd.read_csv(path + 'eng_bowling_against_ind.csv')

ind_batting_vs_pak = pd.read_csv(path + 'ind_batting_against_pak.csv')
ind_bowling_vs_pak = pd.read_csv(path + 'ind_bowling_against_pak.csv')
pak_batting_vs_ind = pd.read_csv(path + 'pak_batting_against_ind.csv')
pak_bowling_vs_ind = pd.read_csv(path + 'pak_bowling_against_ind.csv')

# New teams
ind_batting_vs_sa = pd.read_csv(path + 'ind_batting_against_sa.csv')
ind_bowling_vs_sa = pd.read_csv(path + 'ind_bowling_against_sa.csv')
sa_batting_vs_ind = pd.read_csv(path + 'sa_batting_against_ind.csv')
sa_bowling_vs_ind = pd.read_csv(path + 'sa_bowling_against_ind.csv')

ind_batting_vs_sri = pd.read_csv(path + 'ind_batting_against_sri.csv')
ind_bowling_vs_sri = pd.read_csv(path + 'ind_bowling_against_sri.csv')
sri_batting_vs_ind = pd.read_csv(path + 'sri_batting_against_ind.csv')
sri_bowling_vs_ind = pd.read_csv(path + 'sri_bowling_against_ind.csv')

# Afghanistan team
ind_batting_vs_afg = pd.read_csv(path + 'ind_batting_against_afg.csv')
ind_bowling_vs_afg = pd.read_csv(path + 'ind_bowling_against_afg.csv')
afg_batting_vs_ind = pd.read_csv(path + 'afg_batting_against_ind.csv')
afg_bowling_vs_ind = pd.read_csv(path + 'afg_bowling_against_ind.csv')

stadium_conditions = pd.read_csv(path + 'stadium_conditions.csv')

# Rename stadium conditions columns
stadium_conditions.rename(columns={'Stadium Name': 'stadium_name'}, inplace=True)

# Check if 'Condition' column exists, add if missing
if 'Condition' not in stadium_conditions.columns:
    print("Warning: Column 'Condition' is missing from stadium_conditions DataFrame. Adding default values.")
    stadium_conditions['Condition'] = 'Neutral'  # Default value

# Renaming dictionaries
batting_rename_dict = {
    'Avg': 'avg',
    'Strike Rate': 'strike_rate',
    'Runs': 'runs',
    'Strike rate': 'strike_rate',
    'Not out': 'Not Out'
}

bowling_rename_dict = {
    'Avg': 'avg',
    'Ave': 'avg',
    'Econ': 'econ'
}

# Rename columns to ensure consistency
def rename_columns(df, rename_dict):
    df.rename(columns=rename_dict, inplace=True)
    return df

# Apply renaming
dataframes = [
    ind_batting_vs_aus, ind_bowling_vs_aus, aus_batting_vs_ind, aus_bowling_vs_ind,
    ind_batting_vs_eng, ind_bowling_vs_eng, eng_batting_vs_ind, eng_bowling_vs_ind,
    ind_batting_vs_pak, ind_bowling_vs_pak, pak_batting_vs_ind, pak_bowling_vs_ind,
    ind_batting_vs_sa, ind_bowling_vs_sa, sa_batting_vs_ind, sa_bowling_vs_ind,
    ind_batting_vs_sri, ind_bowling_vs_sri, sri_batting_vs_ind, sri_bowling_vs_ind,
    ind_batting_vs_afg, ind_bowling_vs_afg, afg_batting_vs_ind, afg_bowling_vs_ind
]

for df in dataframes:
    rename_columns(df, batting_rename_dict)
    rename_columns(df, bowling_rename_dict)

# Convert relevant columns to numeric
def convert_columns_to_numeric(df, columns):
    for column in columns:
        if column in df.columns:
            df[column] = pd.to_numeric(df[column], errors='coerce')
        else:
            print(f"Warning: Column '{column}' not found in DataFrame")
    return df

required_batting_columns = ['avg', 'strike_rate', 'runs']
required_bowling_columns = ['avg', 'econ']

# Apply conversion
for df in dataframes:
    convert_columns_to_numeric(df, required_batting_columns)
    convert_columns_to_numeric(df, required_bowling_columns)

# Fill NaN values with relevant random values
def fill_na_with_random(df, columns):
    for column in columns:
        if column in df.columns:
            mean_value = df[column].mean()
            std_value = df[column].std()
            random_values = np.random.normal(mean_value, std_value, df[column].isna().sum())
            df.loc[df[column].isna(), column] = random_values
        else:
            print(f"Warning: Column '{column}' not found in DataFrame")
    return df

# Apply fill_na_with_random
for df in dataframes:
    fill_na_with_random(df, required_batting_columns)
    fill_na_with_random(df, required_bowling_columns)

# Ensure columns exist in the DataFrame
def ensure_columns_exist(df, columns):
    for column in columns:
        if column not in df.columns:
            if column == 'econ':
                df[column] = np.random.normal(4.0, 0.5, len(df))  # Assuming an average economy of 4.0
            else:
                df[column] = np.nan
    return df

# Apply ensure_columns_exist
for df in dataframes:
    ensure_columns_exist(df, required_batting_columns)
    ensure_columns_exist(df, required_bowling_columns)

# Select best 11 players with a balance of 6 batsmen and 5 bowlers, adjusted for stadium conditions
def select_best_11(batting_df, bowling_df, stadium_info):
    # Ensure required columns exist
    batting_df = ensure_columns_exist(batting_df, ['avg', 'strike_rate'])
    bowling_df = ensure_columns_exist(bowling_df, ['avg', 'econ'])

    # Determine stadium preference
    if stadium_info['Condition'].values[0] == 'Batting':
        batting_weight = 1.5
        bowling_weight = 0.75
    elif stadium_info['Condition'].values[0] == 'Bowling':
        batting_weight = 0.75
        bowling_weight = 1.5
    else:
        batting_weight = 1.0
        bowling_weight = 1.0

    # Adjust player metrics based on stadium conditions and add randomness
    batting_df['adjusted_avg'] = batting_df['avg'] * batting_weight * np.random.uniform(0.9, 1.1, len(batting_df))
    bowling_df['adjusted_avg'] = bowling_df['avg'] * bowling_weight * np.random.uniform(0.9, 1.1, len(bowling_df))

    # Select top 6 batsmen based on adjusted average
    best_batsmen = batting_df.sort_values(by='adjusted_avg', ascending=False).head(6)
    # Select top 5 bowlers based on adjusted average
    best_bowlers = bowling_df.sort_values(by='adjusted_avg', ascending=True).head(5)

    # Combine all selected players
    best_11 = pd.concat([best_batsmen, best_bowlers], ignore_index=True)

    # Ensure exactly 11 players
    if len(best_11) < 11:
        # If less than 11 players are found, add more from remaining players
        remaining_players = pd.concat([batting_df, bowling_df]).drop_duplicates()
        additional_players = remaining_players[~remaining_players['Player'].isin(best_11['Player'])].head(11 - len(best_11))
        best_11 = pd.concat([best_11, additional_players], ignore_index=True)

    return best_11

# GUI Implementation
class CricketTeamSelector:
    def __init__(self, root):
        self.root = root
        self.root.title("Cricket Team Selector")
        self.root.geometry("600x400")

        self.team_label = tk.Label(root, text="Select Team:")
        self.team_label.pack()

        self.team_combobox = ttk.Combobox(root, values=["India", "Australia", "England", "Pakistan", "South Africa", "Sri Lanka", "Afghanistan"])
        self.team_combobox.pack()

        self.opponent_label = tk.Label(root, text="Select Opponent Team:")
        self.opponent_label.pack()

        # Ensure "India" is included in the opponent team list
        self.opponent_combobox = ttk.Combobox(root, values=["India", "Australia", "England", "Pakistan", "South Africa", "Sri Lanka", "Afghanistan"])
        self.opponent_combobox.pack()

        self.stadium_label = tk.Label(root, text="Select Stadium:")
        self.stadium_label.pack()

        self.stadium_combobox = ttk.Combobox(root, values=stadium_conditions['stadium_name'].tolist())
        self.stadium_combobox.pack()

        self.submit_button = tk.Button(root, text="Show Best 11", command=self.show_team)
        self.submit_button.pack()

        self.team_output = tk.Text(root, height=15, width=50)
        self.team_output.pack()

    def show_team(self):
        selected_team = self.team_combobox.get()
        opponent_team = self.opponent_combobox.get()
        stadium_name = self.stadium_combobox.get()

        if stadium_name in stadium_conditions['stadium_name'].values:
            stadium_info = stadium_conditions[stadium_conditions['stadium_name'] == stadium_name]
        else:
            self.team_output.delete(1.0, tk.END)
            self.team_output.insert(tk.END, "Invalid stadium selection.")
            return

        # Determine which DataFrame to use based on selected teams
        if selected_team == "India" and opponent_team == "Australia":
            team_batting_data = ind_batting_vs_aus
            team_bowling_data = ind_bowling_vs_aus
        elif selected_team == "Australia" and opponent_team == "India":
            team_batting_data = aus_batting_vs_ind
            team_bowling_data = aus_bowling_vs_ind
        elif selected_team == "India" and opponent_team == "England":
            team_batting_data = ind_batting_vs_eng
            team_bowling_data = ind_bowling_vs_eng
        elif selected_team == "England" and opponent_team == "India":
            team_batting_data = eng_batting_vs_ind
            team_bowling_data = eng_bowling_vs_ind
        elif selected_team == "India" and opponent_team == "Pakistan":
            team_batting_data = ind_batting_vs_pak
            team_bowling_data = ind_bowling_vs_pak
        elif selected_team == "Pakistan" and opponent_team == "India":
            team_batting_data = pak_batting_vs_ind
            team_bowling_data = pak_bowling_vs_ind
        elif selected_team == "India" and opponent_team == "South Africa":
            team_batting_data = ind_batting_vs_sa
            team_bowling_data = ind_bowling_vs_sa
        elif selected_team == "South Africa" and opponent_team == "India":
            team_batting_data = sa_batting_vs_ind
            team_bowling_data = sa_bowling_vs_ind
        elif selected_team == "India" and opponent_team == "Sri Lanka":
            team_batting_data = ind_batting_vs_sri
            team_bowling_data = ind_bowling_vs_sri
        elif selected_team == "Sri Lanka" and opponent_team == "India":
            team_batting_data = sri_batting_vs_ind
            team_bowling_data = sri_bowling_vs_ind
        elif selected_team == "India" and opponent_team == "Afghanistan":
            team_batting_data = ind_batting_vs_afg
            team_bowling_data = ind_bowling_vs_afg
        elif selected_team == "Afghanistan" and opponent_team == "India":
            team_batting_data = afg_batting_vs_ind
            team_bowling_data = afg_bowling_vs_ind
        else:
            self.team_output.delete(1.0, tk.END)
            self.team_output.insert(tk.END, "Invalid team or opponent selection.")
            return

        best_11 = select_best_11(team_batting_data, team_bowling_data, stadium_info)

        self.team_output.delete(1.0, tk.END)

        for _, row in best_11.iterrows():
            player_type = "All-Rounder"
            if row['Player'] in team_batting_data['Player'].values and row['Player'] not in team_bowling_data['Player'].values:
                player_type = "Batsman"
            elif row['Player'] in team_bowling_data['Player'].values and row['Player'] not in team_batting_data['Player'].values:
                player_type = "Bowler"

            player_info = f"Player: {row['Player']} - Type: {player_type}"
            if player_type == "Batsman":
                player_info += f", Strike Rate: {row['strike_rate']:.2f}, Batting Avg: {row['avg']:.2f}"
            elif player_type == "Bowler":
                player_info += f", Bowling Avg: {row['avg']:.2f}, Economy: {row['econ']:.2f}"
            else:  # All-Rounder
                if 'strike_rate' in row and not pd.isna(row['strike_rate']):
                    player_info += f", Strike Rate: {row['strike_rate']:.2f}"
                if 'avg' in row and not pd.isna(row['avg']):
                    player_info += f", Batting Avg: {row['avg']:.2f}"
                if 'avg' in row and not pd.isna(row['avg']):
                    player_info += f", Bowling Avg: {row['avg']:.2f}"
                if 'econ' in row and not pd.isna(row['econ']):
                    player_info += f", Economy: {row['econ']:.2f}"

            self.team_output.insert(tk.END, f"{player_info}\n")

root = tk.Tk()
app = CricketTeamSelector(root)
root.mainloop()


till here afg added!

In [ ]:
import pandas as pd
import tkinter as tk
from tkinter import ttk
import numpy as np

# Load CSV files
path = r'E:\Major project\\'

# Existing teams
ind_batting_vs_aus = pd.read_csv(path + 'ind_batting_against_aus.csv')
ind_bowling_vs_aus = pd.read_csv(path + 'ind_bowling_against_aus.csv')
aus_batting_vs_ind = pd.read_csv(path + 'aus_batting_against_ind.csv')
aus_bowling_vs_ind = pd.read_csv(path + 'aus_bowling_against_ind.csv')

ind_batting_vs_eng = pd.read_csv(path + 'ind_batting_against_eng.csv')
ind_bowling_vs_eng = pd.read_csv(path + 'ind_bowling_against_eng.csv')
eng_batting_vs_ind = pd.read_csv(path + 'eng_batting_against_ind.csv')
eng_bowling_vs_ind = pd.read_csv(path + 'eng_bowling_against_ind.csv')

ind_batting_vs_pak = pd.read_csv(path + 'ind_batting_against_pak.csv')
ind_bowling_vs_pak = pd.read_csv(path + 'ind_bowling_against_pak.csv')
pak_batting_vs_ind = pd.read_csv(path + 'pak_batting_against_ind.csv')
pak_bowling_vs_ind = pd.read_csv(path + 'pak_bowling_against_ind.csv')

ind_batting_vs_sa = pd.read_csv(path + 'ind_batting_against_sa.csv')
ind_bowling_vs_sa = pd.read_csv(path + 'ind_bowling_against_sa.csv')
sa_batting_vs_ind = pd.read_csv(path + 'sa_batting_against_ind.csv')
sa_bowling_vs_ind = pd.read_csv(path + 'sa_bowling_against_ind.csv')

ind_batting_vs_sri = pd.read_csv(path + 'ind_batting_against_sri.csv')
ind_bowling_vs_sri = pd.read_csv(path + 'ind_bowling_against_sri.csv')
sri_batting_vs_ind = pd.read_csv(path + 'sri_batting_against_ind.csv')
sri_bowling_vs_ind = pd.read_csv(path + 'sri_bowling_against_ind.csv')

ind_batting_vs_afg = pd.read_csv(path + 'ind_batting_against_afg.csv')
ind_bowling_vs_afg = pd.read_csv(path + 'ind_bowling_against_afg.csv')
afg_batting_vs_ind = pd.read_csv(path + 'afg_batting_against_ind.csv')
afg_bowling_vs_ind = pd.read_csv(path + 'afg_bowling_against_ind.csv')

# New Zealand team
ind_batting_vs_nz = pd.read_csv(path + 'ind_batting_against_nz.csv')
ind_bowling_vs_nz = pd.read_csv(path + 'ind_bowling_against_nz.csv')
nz_batting_vs_ind = pd.read_csv(path + 'nz_batting_against_ind.csv')
nz_bowling_vs_ind = pd.read_csv(path + 'nz_bowling_against_ind.csv')

stadium_conditions = pd.read_csv(path + 'stadium_conditions.csv')

# Rename stadium conditions columns
stadium_conditions.rename(columns={'Stadium Name': 'stadium_name'}, inplace=True)

# Check if 'Condition' column exists, add if missing
if 'Condition' not in stadium_conditions.columns:
    print("Warning: Column 'Condition' is missing from stadium_conditions DataFrame. Adding default values.")
    stadium_conditions['Condition'] = 'Neutral'  # Default value

# Renaming dictionaries
batting_rename_dict = {
    'Avg': 'avg',
    'Strike Rate': 'strike_rate',
    'Runs': 'runs',
    'Strike rate': 'strike_rate',
    'Not out': 'Not Out'
}

bowling_rename_dict = {
    'Avg': 'avg',
    'Ave': 'avg',
    'Econ': 'econ'
}

# Rename columns to ensure consistency
def rename_columns(df, rename_dict):
    df.rename(columns=rename_dict, inplace=True)
    return df

# Apply renaming
dataframes = [
    ind_batting_vs_aus, ind_bowling_vs_aus, aus_batting_vs_ind, aus_bowling_vs_ind,
    ind_batting_vs_eng, ind_bowling_vs_eng, eng_batting_vs_ind, eng_bowling_vs_ind,
    ind_batting_vs_pak, ind_bowling_vs_pak, pak_batting_vs_ind, pak_bowling_vs_ind,
    ind_batting_vs_sa, ind_bowling_vs_sa, sa_batting_vs_ind, sa_bowling_vs_ind,
    ind_batting_vs_sri, ind_bowling_vs_sri, sri_batting_vs_ind, sri_bowling_vs_ind,
    ind_batting_vs_afg, ind_bowling_vs_afg, afg_batting_vs_ind, afg_bowling_vs_ind,
    ind_batting_vs_nz, ind_bowling_vs_nz, nz_batting_vs_ind, nz_bowling_vs_ind
]

for df in dataframes:
    rename_columns(df, batting_rename_dict)
    rename_columns(df, bowling_rename_dict)

# Convert relevant columns to numeric
def convert_columns_to_numeric(df, columns):
    for column in columns:
        if column in df.columns:
            df[column] = pd.to_numeric(df[column], errors='coerce')
        else:
            print(f"Warning: Column '{column}' not found in DataFrame")
    return df

required_batting_columns = ['avg', 'strike_rate', 'runs']
required_bowling_columns = ['avg', 'econ']

# Apply conversion
for df in dataframes:
    convert_columns_to_numeric(df, required_batting_columns)
    convert_columns_to_numeric(df, required_bowling_columns)

# Fill NaN values with relevant random values
def fill_na_with_random(df, columns):
    for column in columns:
        if column in df.columns:
            mean_value = df[column].mean()
            std_value = df[column].std()
            random_values = np.random.normal(mean_value, std_value, df[column].isna().sum())
            df.loc[df[column].isna(), column] = random_values
        else:
            print(f"Warning: Column '{column}' not found in DataFrame")
    return df

# Apply fill_na_with_random
for df in dataframes:
    fill_na_with_random(df, required_batting_columns)
    fill_na_with_random(df, required_bowling_columns)

# Ensure columns exist in the DataFrame
def ensure_columns_exist(df, columns):
    for column in columns:
        if column not in df.columns:
            if column == 'econ':
                df[column] = np.random.normal(4.0, 0.5, len(df))  # Assuming an average economy of 4.0
            else:
                df[column] = np.nan
    return df

# Apply ensure_columns_exist
for df in dataframes:
    ensure_columns_exist(df, required_batting_columns)
    ensure_columns_exist(df, required_bowling_columns)

# Select best 11 players with a balance of 6 batsmen and 5 bowlers, adjusted for stadium conditions
def select_best_11(batting_df, bowling_df, stadium_info):
    # Ensure required columns exist
    batting_df = ensure_columns_exist(batting_df, ['avg', 'strike_rate'])
    bowling_df = ensure_columns_exist(bowling_df, ['avg', 'econ'])

    # Determine stadium preference
    if stadium_info['Condition'].values[0] == 'Batting':
        batting_weight = 1.5
        bowling_weight = 0.75
    elif stadium_info['Condition'].values[0] == 'Bowling':
        batting_weight = 0.75
        bowling_weight = 1.5
    else:
        batting_weight = 1.0
        bowling_weight = 1.0

    # Adjust player metrics based on stadium conditions and add randomness
    batting_df['adjusted_avg'] = batting_df['avg'] * batting_weight * np.random.uniform(0.9, 1.1, len(batting_df))
    bowling_df['adjusted_avg'] = bowling_df['avg'] * bowling_weight * np.random.uniform(0.9, 1.1, len(bowling_df))

    # Select top 6 batsmen based on adjusted average
    best_batsmen = batting_df.sort_values(by='adjusted_avg', ascending=False).head(6)
    # Select top 5 bowlers based on adjusted average
    best_bowlers = bowling_df.sort_values(by='adjusted_avg', ascending=True).head(5)

    # Combine all selected players
    best_11 = pd.concat([best_batsmen, best_bowlers], ignore_index=True).drop_duplicates(subset='Player')

    # Ensure exactly 11 players
    if len(best_11) < 11:
        # If less than 11 players are found, add more from remaining players
        remaining_players = pd.concat([batting_df, bowling_df]).drop_duplicates(subset='Player')
        additional_players = remaining_players[~remaining_players['Player'].isin(best_11['Player'])].head(11 - len(best_11))
        best_11 = pd.concat([best_11, additional_players], ignore_index=True).drop_duplicates(subset='Player')

    return best_11

# GUI Implementation
class CricketTeamSelector:
    def __init__(self, root):
        self.root = root
        self.root.title("Cricket Team Selector")
        self.root.geometry("600x400")

        self.team_label = tk.Label(root, text="Select Team:")
        self.team_label.pack()

        self.team_combobox = ttk.Combobox(root, values=["India", "Australia", "England", "Pakistan", "South Africa", "Sri Lanka", "Afghanistan", "New Zealand"])
        self.team_combobox.pack()

        self.opponent_label = tk.Label(root, text="Select Opponent Team:")
        self.opponent_label.pack()

        # Ensure "India" is included in the opponent team list
        self.opponent_combobox = ttk.Combobox(root, values=["India", "Australia", "England", "Pakistan", "South Africa", "Sri Lanka", "Afghanistan", "New Zealand"])
        self.opponent_combobox.pack()

        self.stadium_label = tk.Label(root, text="Select Stadium:")
        self.stadium_label.pack()

        self.stadium_combobox = ttk.Combobox(root, values=stadium_conditions['stadium_name'].tolist())
        self.stadium_combobox.pack()

        self.submit_button = tk.Button(root, text="Show Best 11", command=self.show_team)
        self.submit_button.pack()

        self.team_output = tk.Text(root, height=15, width=50)
        self.team_output.pack()

    def show_team(self):
        selected_team = self.team_combobox.get()
        opponent_team = self.opponent_combobox.get()
        stadium_name = self.stadium_combobox.get()

        if stadium_name in stadium_conditions['stadium_name'].values:
            stadium_info = stadium_conditions[stadium_conditions['stadium_name'] == stadium_name]
        else:
            self.team_output.delete(1.0, tk.END)
            self.team_output.insert(tk.END, "Invalid stadium selection.")
            return

        # Determine which DataFrame to use based on selected teams
        team_batting_data = None
        team_bowling_data = None

        if selected_team == "India" and opponent_team == "Australia":
            team_batting_data = ind_batting_vs_aus
            team_bowling_data = ind_bowling_vs_aus
        elif selected_team == "Australia" and opponent_team == "India":
            team_batting_data = aus_batting_vs_ind
            team_bowling_data = aus_bowling_vs_ind
        elif selected_team == "India" and opponent_team == "England":
            team_batting_data = ind_batting_vs_eng
            team_bowling_data = ind_bowling_vs_eng
        elif selected_team == "England" and opponent_team == "India":
            team_batting_data = eng_batting_vs_ind
            team_bowling_data = eng_bowling_vs_ind
        elif selected_team == "India" and opponent_team == "Pakistan":
            team_batting_data = ind_batting_vs_pak
            team_bowling_data = ind_bowling_vs_pak
        elif selected_team == "Pakistan" and opponent_team == "India":
            team_batting_data = pak_batting_vs_ind
            team_bowling_data = pak_bowling_vs_ind
        elif selected_team == "India" and opponent_team == "South Africa":
            team_batting_data = ind_batting_vs_sa
            team_bowling_data = ind_bowling_vs_sa
        elif selected_team == "South Africa" and opponent_team == "India":
            team_batting_data = sa_batting_vs_ind
            team_bowling_data = sa_bowling_vs_ind
        elif selected_team == "India" and opponent_team == "Sri Lanka":
            team_batting_data = ind_batting_vs_sri
            team_bowling_data = ind_bowling_vs_sri
        elif selected_team == "Sri Lanka" and opponent_team == "India":
            team_batting_data = sri_batting_vs_ind
            team_bowling_data = sri_bowling_vs_ind
        elif selected_team == "India" and opponent_team == "Afghanistan":
            team_batting_data = ind_batting_vs_afg
            team_bowling_data = ind_bowling_vs_afg
        elif selected_team == "Afghanistan" and opponent_team == "India":
            team_batting_data = afg_batting_vs_ind
            team_bowling_data = afg_bowling_vs_ind
        elif selected_team == "India" and opponent_team == "New Zealand":
            team_batting_data = ind_batting_vs_nz
            team_bowling_data = ind_bowling_vs_nz
        elif selected_team == "New Zealand" and opponent_team == "India":
            team_batting_data = nz_batting_vs_ind
            team_bowling_data = nz_bowling_vs_ind
        else:
            self.team_output.delete(1.0, tk.END)
            self.team_output.insert(tk.END, "Invalid team or opponent selection.")
            return

        best_11 = select_best_11(team_batting_data, team_bowling_data, stadium_info)

        self.team_output.delete(1.0, tk.END)

        for _, row in best_11.iterrows():
            player_type = "All-Rounder"
            if row['Player'] in team_batting_data['Player'].values and row['Player'] not in team_bowling_data['Player'].values:
                player_type = "Batsman"
            elif row['Player'] in team_bowling_data['Player'].values and row['Player'] not in team_batting_data['Player'].values:
                player_type = "Bowler"

            player_info = f"Player: {row['Player']} - Type: {player_type}"
            if player_type == "Batsman":
                player_info += f", Strike Rate: {row['strike_rate']:.2f}, Batting Avg: {row['avg']:.2f}"
            elif player_type == "Bowler":
                player_info += f", Bowling Avg: {row['avg']:.2f}, Economy: {row['econ']:.2f}"
            else:  # All-Rounder
                if 'strike_rate' in row and not pd.isna(row['strike_rate']):
                    player_info += f", Strike Rate: {row['strike_rate']:.2f}"
                if 'avg' in row and not pd.isna(row['avg']):
                    player_info += f", Batting Avg: {row['avg']:.2f}"
                if 'avg' in row and not pd.isna(row['avg']):
                    player_info += f", Bowling Avg: {row['avg']:.2f}"
                if 'econ' in row and not pd.isna(row['econ']):
                    player_info += f", Economy: {row['econ']:.2f}"

            self.team_output.insert(tk.END, f"{player_info}\n")

root = tk.Tk()
app = CricketTeamSelector(root)
root.mainloop()


Till here the problem has been resolved!(wi and Ban pending)

In [ ]:
import pandas as pd
import tkinter as tk
from tkinter import ttk
import numpy as np

# Load CSV files
path = r'E:\Major project\\'

# Existing teams
ind_batting_vs_aus = pd.read_csv(path + 'ind_batting_against_aus.csv')
ind_bowling_vs_aus = pd.read_csv(path + 'ind_bowling_against_aus.csv')
aus_batting_vs_ind = pd.read_csv(path + 'aus_batting_against_ind.csv')
aus_bowling_vs_ind = pd.read_csv(path + 'aus_bowling_against_ind.csv')

ind_batting_vs_eng = pd.read_csv(path + 'ind_batting_against_eng.csv')
ind_bowling_vs_eng = pd.read_csv(path + 'ind_bowling_against_eng.csv')
eng_batting_vs_ind = pd.read_csv(path + 'eng_batting_against_ind.csv')
eng_bowling_vs_ind = pd.read_csv(path + 'eng_bowling_against_ind.csv')

ind_batting_vs_pak = pd.read_csv(path + 'ind_batting_against_pak.csv')
ind_bowling_vs_pak = pd.read_csv(path + 'ind_bowling_against_pak.csv')
pak_batting_vs_ind = pd.read_csv(path + 'pak_batting_against_ind.csv')
pak_bowling_vs_ind = pd.read_csv(path + 'pak_bowling_against_ind.csv')

ind_batting_vs_sa = pd.read_csv(path + 'ind_batting_against_sa.csv')
ind_bowling_vs_sa = pd.read_csv(path + 'ind_bowling_against_sa.csv')
sa_batting_vs_ind = pd.read_csv(path + 'sa_batting_against_ind.csv')
sa_bowling_vs_ind = pd.read_csv(path + 'sa_bowling_against_ind.csv')

ind_batting_vs_sri = pd.read_csv(path + 'ind_batting_against_sri.csv')
ind_bowling_vs_sri = pd.read_csv(path + 'ind_bowling_against_sri.csv')
sri_batting_vs_ind = pd.read_csv(path + 'sri_batting_against_ind.csv')
sri_bowling_vs_ind = pd.read_csv(path + 'sri_bowling_against_ind.csv')

ind_batting_vs_afg = pd.read_csv(path + 'ind_batting_against_afg.csv')
ind_bowling_vs_afg = pd.read_csv(path + 'ind_bowling_against_afg.csv')
afg_batting_vs_ind = pd.read_csv(path + 'afg_batting_against_ind.csv')
afg_bowling_vs_ind = pd.read_csv(path + 'afg_bowling_against_ind.csv')

ind_batting_vs_nz = pd.read_csv(path + 'ind_batting_against_nz.csv')
ind_bowling_vs_nz = pd.read_csv(path + 'ind_bowling_against_nz.csv')
nz_batting_vs_ind = pd.read_csv(path + 'nz_batting_against_ind.csv')
nz_bowling_vs_ind = pd.read_csv(path + 'nz_bowling_against_ind.csv')

# Additional teams
ind_batting_vs_wi = pd.read_csv(path + 'ind_batting_against_wi.csv')
ind_bowling_vs_wi = pd.read_csv(path + 'ind_bowling_against_wi.csv')
wi_batting_vs_ind = pd.read_csv(path + 'wi_batting_against_ind.csv')
wi_bowling_vs_ind = pd.read_csv(path + 'wi_bowling_against_ind.csv')

ind_batting_vs_ban = pd.read_csv(path + 'ind_batting_against_ban.csv')
ind_bowling_vs_ban = pd.read_csv(path + 'ind_bowling_against_ban.csv')
ban_batting_vs_ind = pd.read_csv(path + 'ban_batting_against_ind.csv')
ban_bowling_vs_ind = pd.read_csv(path + 'ban_bowling_against_ind.csv')

stadium_conditions = pd.read_csv(path + 'stadium_conditions.csv')

# Rename stadium conditions columns
stadium_conditions.rename(columns={'Stadium Name': 'stadium_name'}, inplace=True)

# Check if 'Condition' column exists, add if missing
if 'Condition' not in stadium_conditions.columns:
    print("Warning: Column 'Condition' is missing from stadium_conditions DataFrame. Adding default values.")
    stadium_conditions['Condition'] = 'Neutral'  # Default value

# Renaming dictionaries
batting_rename_dict = {
    'Avg': 'avg',
    'Strike Rate': 'strike_rate',
    'Runs': 'runs',
    'Strike rate': 'strike_rate',
    'Not out': 'Not Out'
}

bowling_rename_dict = {
    'Avg': 'avg',
    'Ave': 'avg',
    'Econ': 'econ'
}

# Rename columns to ensure consistency
def rename_columns(df, rename_dict):
    df.rename(columns=rename_dict, inplace=True)
    return df

# Apply renaming
dataframes = [
    ind_batting_vs_aus, ind_bowling_vs_aus, aus_batting_vs_ind, aus_bowling_vs_ind,
    ind_batting_vs_eng, ind_bowling_vs_eng, eng_batting_vs_ind, eng_bowling_vs_ind,
    ind_batting_vs_pak, ind_bowling_vs_pak, pak_batting_vs_ind, pak_bowling_vs_ind,
    ind_batting_vs_sa, ind_bowling_vs_sa, sa_batting_vs_ind, sa_bowling_vs_ind,
    ind_batting_vs_sri, ind_bowling_vs_sri, sri_batting_vs_ind, sri_bowling_vs_ind,
    ind_batting_vs_afg, ind_bowling_vs_afg, afg_batting_vs_ind, afg_bowling_vs_ind,
    ind_batting_vs_nz, ind_bowling_vs_nz, nz_batting_vs_ind, nz_bowling_vs_ind,
    ind_batting_vs_wi, ind_bowling_vs_wi, wi_batting_vs_ind, wi_bowling_vs_ind,
    ind_batting_vs_ban, ind_bowling_vs_ban, ban_batting_vs_ind, ban_bowling_vs_ind
]

for df in dataframes:
    rename_columns(df, batting_rename_dict)
    rename_columns(df, bowling_rename_dict)

# Convert relevant columns to numeric
def convert_columns_to_numeric(df, columns):
    for column in columns:
        if column in df.columns:
            df[column] = pd.to_numeric(df[column], errors='coerce')
        else:
            print(f"Warning: Column '{column}' not found in DataFrame")
    return df

required_batting_columns = ['avg', 'strike_rate', 'runs']
required_bowling_columns = ['avg', 'econ']

# Apply conversion
for df in dataframes:
    convert_columns_to_numeric(df, required_batting_columns)
    convert_columns_to_numeric(df, required_bowling_columns)

# Fill NaN values with relevant random values
def fill_na_with_random(df, columns):
    for column in columns:
        if column in df.columns:
            mean_value = df[column].mean()
            std_value = df[column].std()
            random_values = np.random.normal(mean_value, std_value, df[column].isna().sum())
            df.loc[df[column].isna(), column] = random_values
        else:
            print(f"Warning: Column '{column}' not found in DataFrame")
    return df

# Apply fill_na_with_random
for df in dataframes:
    fill_na_with_random(df, required_batting_columns)
    fill_na_with_random(df, required_bowling_columns)

# Ensure columns exist in the DataFrame
def ensure_columns_exist(df, columns):
    for column in columns:
        if column not in df.columns:
            if column == 'econ':
                df[column] = np.random.normal(4.0, 0.5, len(df))  # Assuming an average economy of 4.0
            else:
                df[column] = np.nan
    return df

# Apply ensure_columns_exist
for df in dataframes:
    ensure_columns_exist(df, required_batting_columns)
    ensure_columns_exist(df, required_bowling_columns)

# Select best 11 players with a balance of 6 batsmen and 5 bowlers, adjusted for stadium conditions
def select_best_11(batting_df, bowling_df, stadium_info):
    # Ensure required columns exist
    batting_df = ensure_columns_exist(batting_df, ['avg', 'strike_rate'])
    bowling_df = ensure_columns_exist(bowling_df, ['avg', 'econ'])

    # Determine stadium preference
    if stadium_info['Condition'].values[0] == 'Batting':
        batting_weight = 1.5
        bowling_weight = 0.75
    elif stadium_info['Condition'].values[0] == 'Bowling':
        batting_weight = 0.75
        bowling_weight = 1.5
    else:
        batting_weight = 1.0
        bowling_weight = 1.0

    # Adjust player metrics based on stadium conditions and add randomness
    batting_df['adjusted_avg'] = batting_df['avg'] * batting_weight * np.random.uniform(0.9, 1.1, len(batting_df))
    bowling_df['adjusted_avg'] = bowling_df['avg'] * bowling_weight * np.random.uniform(0.9, 1.1, len(bowling_df))

    # Select top 6 batsmen based on adjusted average
    best_batsmen = batting_df.sort_values(by='adjusted_avg', ascending=False).head(6)
    # Select top 5 bowlers based on adjusted average
    best_bowlers = bowling_df.sort_values(by='adjusted_avg', ascending=True).head(5)

    # Combine all selected players
    best_11 = pd.concat([best_batsmen, best_bowlers], ignore_index=True).drop_duplicates(subset='Player')

    # Ensure exactly 11 players
    if len(best_11) < 11:
        # If less than 11 players are found, add more from remaining players
        remaining_players = pd.concat([batting_df, bowling_df]).drop_duplicates(subset='Player')
        additional_players = remaining_players[~remaining_players['Player'].isin(best_11['Player'])].head(11 - len(best_11))
        best_11 = pd.concat([best_11, additional_players], ignore_index=True).drop_duplicates(subset='Player')

    return best_11

# GUI Implementation
class CricketTeamSelector:
    def __init__(self, root):
        self.root = root
        self.root.title("Cricket Team Selector")
        self.root.geometry("600x400")

        self.team_label = tk.Label(root, text="Select Team:")
        self.team_label.pack()

        self.team_combobox = ttk.Combobox(root, values=["India", "Australia", "England", "Pakistan", "South Africa", "Sri Lanka", "Afghanistan", "New Zealand", "West Indies", "Bangladesh"])
        self.team_combobox.pack()

        self.opponent_label = tk.Label(root, text="Select Opponent Team:")
        self.opponent_label.pack()

        # Ensure "India" is included in the opponent team list
        self.opponent_combobox = ttk.Combobox(root, values=["India", "Australia", "England", "Pakistan", "South Africa", "Sri Lanka", "Afghanistan", "New Zealand", "West Indies", "Bangladesh"])
        self.opponent_combobox.pack()

        self.stadium_label = tk.Label(root, text="Select Stadium:")
        self.stadium_label.pack()

        self.stadium_combobox = ttk.Combobox(root, values=stadium_conditions['stadium_name'].tolist())
        self.stadium_combobox.pack()

        self.submit_button = tk.Button(root, text="Show Best 11", command=self.show_team)
        self.submit_button.pack()

        self.team_output = tk.Text(root, height=15, width=50)
        self.team_output.pack()

    def show_team(self):
        selected_team = self.team_combobox.get()
        opponent_team = self.opponent_combobox.get()
        stadium_name = self.stadium_combobox.get()

        if stadium_name in stadium_conditions['stadium_name'].values:
            stadium_info = stadium_conditions[stadium_conditions['stadium_name'] == stadium_name]
        else:
            self.team_output.delete(1.0, tk.END)
            self.team_output.insert(tk.END, "Invalid stadium selection.")
            return

        # Determine which DataFrame to use based on selected teams
        team_batting_data = None
        team_bowling_data = None

        if selected_team == "India" and opponent_team == "Australia":
            team_batting_data = ind_batting_vs_aus
            team_bowling_data = ind_bowling_vs_aus
        elif selected_team == "Australia" and opponent_team == "India":
            team_batting_data = aus_batting_vs_ind
            team_bowling_data = aus_bowling_vs_ind
        elif selected_team == "India" and opponent_team == "England":
            team_batting_data = ind_batting_vs_eng
            team_bowling_data = ind_bowling_vs_eng
        elif selected_team == "England" and opponent_team == "India":
            team_batting_data = eng_batting_vs_ind
            team_bowling_data = eng_bowling_vs_ind
        elif selected_team == "India" and opponent_team == "Pakistan":
            team_batting_data = ind_batting_vs_pak
            team_bowling_data = ind_bowling_vs_pak
        elif selected_team == "Pakistan" and opponent_team == "India":
            team_batting_data = pak_batting_vs_ind
            team_bowling_data = pak_bowling_vs_ind
        elif selected_team == "India" and opponent_team == "South Africa":
            team_batting_data = ind_batting_vs_sa
            team_bowling_data = ind_bowling_vs_sa
        elif selected_team == "South Africa" and opponent_team == "India":
            team_batting_data = sa_batting_vs_ind
            team_bowling_data = sa_bowling_vs_ind
        elif selected_team == "India" and opponent_team == "Sri Lanka":
            team_batting_data = ind_batting_vs_sri
            team_bowling_data = ind_bowling_vs_sri
        elif selected_team == "Sri Lanka" and opponent_team == "India":
            team_batting_data = sri_batting_vs_ind
            team_bowling_data = sri_bowling_vs_ind
        elif selected_team == "India" and opponent_team == "Afghanistan":
            team_batting_data = ind_batting_vs_afg
            team_bowling_data = ind_bowling_vs_afg
        elif selected_team == "Afghanistan" and opponent_team == "India":
            team_batting_data = afg_batting_vs_ind
            team_bowling_data = afg_bowling_vs_ind
        elif selected_team == "India" and opponent_team == "New Zealand":
            team_batting_data = ind_batting_vs_nz
            team_bowling_data = ind_bowling_vs_nz
        elif selected_team == "New Zealand" and opponent_team == "India":
            team_batting_data = nz_batting_vs_ind
            team_bowling_data = nz_bowling_vs_ind
        elif selected_team == "India" and opponent_team == "West Indies":
            team_batting_data = ind_batting_vs_wi
            team_bowling_data = ind_bowling_vs_wi
        elif selected_team == "West Indies" and opponent_team == "India":
            team_batting_data = wi_batting_vs_ind
            team_bowling_data = wi_bowling_vs_ind
        elif selected_team == "India" and opponent_team == "Bangladesh":
            team_batting_data = ind_batting_vs_ban
            team_bowling_data = ind_bowling_vs_ban
        elif selected_team == "Bangladesh" and opponent_team == "India":
            team_batting_data = ban_batting_vs_ind
            team_bowling_data = ban_bowling_vs_ind
        else:
            self.team_output.delete(1.0, tk.END)
            self.team_output.insert(tk.END, "Invalid team or opponent selection.")
            return

        best_11 = select_best_11(team_batting_data, team_bowling_data, stadium_info)

        self.team_output.delete(1.0, tk.END)

        for _, row in best_11.iterrows():
            player_type = "All-Rounder"
            if row['Player'] in team_batting_data['Player'].values and row['Player'] not in team_bowling_data['Player'].values:
                player_type = "Batsman"
            elif row['Player'] in team_bowling_data['Player'].values and row['Player'] not in team_batting_data['Player'].values:
                player_type = "Bowler"

            player_info = f"Player: {row['Player']} - Type: {player_type}"
            if player_type == "Batsman":
                player_info += f", Strike Rate: {row['strike_rate']:.2f}, Batting Avg: {row['avg']:.2f}"
            elif player_type == "Bowler":
                player_info += f", Bowling Avg: {row['avg']:.2f}, Economy: {row['econ']:.2f}"
            else:  # All-Rounder
                if 'strike_rate' in row and not pd.isna(row['strike_rate']):
                    player_info += f", Strike Rate: {row['strike_rate']:.2f}"
                if 'avg' in row and not pd.isna(row['avg']):
                    player_info += f", Batting Avg: {row['avg']:.2f}"
                if 'avg' in row and not pd.isna(row['avg']):
                    player_info += f", Bowling Avg: {row['avg']:.2f}"
                if 'econ' in row and not pd.isna(row['econ']):
                    player_info += f", Economy: {row['econ']:.2f}"

            self.team_output.insert(tk.END, f"{player_info}\n")

root = tk.Tk()
app = CricketTeamSelector(root)
root.mainloop()


completed all ind vs opponent team !

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Sample data for demonstration purposes (replace with actual data loading)
ind_batting_vs_aus = pd.DataFrame({
    'Player': ['Player1', 'Player2', 'Player3'],
    'avg_bat': [45.0, 30.0, 25.0],
    'strike_rate_bat': [90.0, 85.0, 75.0]
})

ind_bowling_vs_aus = pd.DataFrame({
    'Player': ['Player1', 'Player2', 'Player3'],
    'avg_bowl': [25.0, 30.0, 20.0],
    'econ_bowl': [4.5, 5.0, 3.8]
})

aus_batting_vs_ind = pd.DataFrame({
    'Player': ['Player4', 'Player5', 'Player6'],
    'avg_bat': [40.0, 35.0, 20.0],
    'strike_rate_bat': [80.0, 70.0, 65.0]
})

aus_bowling_vs_ind = pd.DataFrame({
    'Player': ['Player4', 'Player5', 'Player6'],
    'avg_bowl': [30.0, 27.0, 22.0],
    'econ_bowl': [5.0, 4.8, 4.2]
})

# Function to prepare dataset
def prepare_dataset_for_rf(batting_df, bowling_df, result):
    combined_df = pd.merge(batting_df, bowling_df, on='Player', suffixes=('_bat', '_bowl'))
    combined_df['result'] = result
    return combined_df[['avg_bat', 'strike_rate_bat', 'avg_bowl', 'econ_bowl', 'result']]

# Preparing all match data
all_match_data = pd.DataFrame()

for (team_batting, team_bowling, opponent_batting, opponent_bowling) in [
    (ind_batting_vs_aus, ind_bowling_vs_aus, aus_batting_vs_ind, aus_bowling_vs_ind),
    (aus_batting_vs_ind, aus_bowling_vs_ind, ind_batting_vs_aus, ind_bowling_vs_aus)
]:
    ind_win = prepare_dataset_for_rf(team_batting, team_bowling, 1)
    opp_win = prepare_dataset_for_rf(opponent_batting, opponent_bowling, 0)
    all_match_data = pd.concat([all_match_data, ind_win, opp_win], ignore_index=True)

# Split dataset into features and labels
X = all_match_data.drop('result', axis=1)
y = all_match_data['result']

# Train Random Forest model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Evaluate model
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Random Forest Model Accuracy: {accuracy:.2f}")

# Function to prepare input data for prediction
def prepare_input_for_prediction(input_data):
    input_data = input_data.rename(columns={
        'econ': 'econ_bowl',
        'strike_rate': 'strike_rate_bat'
    })
    return input_data

# Example input data for prediction
match_input = pd.DataFrame({
    'avg_bat': [45.0],
    'strike_rate': [90.0],
    'avg_bowl': [25.0],
    'econ': [4.5]
})

# Prepare the input data with correct feature names
match_input = prepare_input_for_prediction(match_input)

# Predict using the Random Forest model
try:
    win_prob = rf_model.predict_proba(match_input)[0][1]
    print(f"Winning Probability: {win_prob:.2f}")
except ValueError as e:
    print(f"Error in prediction: {e}")


Random Forest Model Accuracy: 0.33
Winning Probability: 0.64
